# Data preprocessing of the raw data with 10000 Poker Hand History files
In order to keep things simple, I am not attaching all the 10000 files of raw data to this Github.

The Poker Hand History File Format Specification is described at this link https://phh.readthedocs.io/en/stable/.
For the poker dataset, I am using this link https://zenodo.org/records/17136841.
I downloaded the attached file "poker-hand-histories.zip".
As explained on the link, the ZIP contains many files in the poker hand history (PHH) format.
I extracted the ZIP and focused on the folder "pluribus". 
It contains 10.000 PHH files, and each file corresponds to 1 game between 5 human professional poker players and 1 poker AI named "Pluribus".

If you need the raw data, you need to download the ZIP and move the content of the folder "pluribus" to the Folder Path "folder_path" defined below

## Folder path of the raw data
This is the path where are located the 10000 Poker Hand History files that we want to preprocess to transform into an easy to read CSV file

In [1]:
folder_path = 'C:\\Users\\vince\\School\\Capstone\\poker_github\\data\\phh\\pluribus\\';

## Common attributes for all the 10000 Poker games
- variant : NT. The poker variant is No-limit Texas hold ‘em
- ante : 0. The poker ante is 0
- small_blind : 50. The amount for the small blind is 50
- big_blind : 100. The amount for the big blind is 100
- min_bet : 100. The minimum bet is 100
- starting_stack : 10000. The starting stack amount is 10000

# Load required libraries

In [2]:
import pandas as pd
import numpy as np
import os
from phevaluator.evaluator import evaluate_cards
import warnings
warnings.filterwarnings('ignore')

# Functions

## Function readFilePHH
This function reads a Poker Hand History file representing one poker game between 6 players
and returns 5 dataframes : one about the game general information, 
and one for each phase of the game : preflop, flop, turn and river

In [3]:
def readFilePHH(file_path):
    """
    This function reads a Poker Hand History file representing one poker game between 6 players
    and returns 5 dataframes : one about the game general information, 
    and one for each phase of the game : preflop, flop, turn and river
    
    Arguments
    ---------
    file_path: string
           The path of the PHH file

    Returns
    --------
    df_game_info: pd.DataFrame
         Array representing one game general information
    df_game_actions_preflop: pd.DataFrame
         Array about the actions played during the preflop phase of one game
    df_game_actions_flop: pd.DataFrame
         Array about the actions played during the flop phase of one game
    df_game_actions_turn: pd.DataFrame
         Array about the actions played during the turn phase of one game
    df_game_actions_river: pd.DataFrame
         Array about the actions played during the river phase of one game
    """
    file_path = file_path.replace("\\", "\\\\")

    # Create a dictionary
    dict_game_info = {}
    dict_game_actions_preflop = {}
    dict_game_actions_flop = {}
    dict_game_actions_turn = {}
    dict_game_actions_river = {}
    game_id = file_path[71:].replace("\\", "_").replace(".phh", "").strip()
    dict_game_info['game_id'] = game_id
    dict_game_actions_preflop['game_id'] = game_id
    dict_game_actions_flop['game_id'] = game_id
    dict_game_actions_turn['game_id'] = game_id
    dict_game_actions_river['game_id'] = game_id
    try:
        with open(file_path, "r") as file:
            lines = file.readlines()
            for line in lines:
                column_name = line.split('=')[0].strip()
                column_value = line.split('=')[1].strip()

                # conditions on columns
                if column_name == 'variant':
                    column_value = column_value.replace("'", "")
                if column_name == 'antes':
                    column_value = column_value.replace("[", "").replace("]", "")
                    dict_game_info['p1_ante'] = column_value.split(',')[0].strip()
                    dict_game_info['p2_ante'] = column_value.split(',')[1].strip()
                    dict_game_info['p3_ante'] = column_value.split(',')[2].strip()
                    dict_game_info['p4_ante'] = column_value.split(',')[3].strip()
                    dict_game_info['p5_ante'] = column_value.split(',')[4].strip()
                    dict_game_info['p6_ante'] = column_value.split(',')[5].strip()
                if column_name == 'blinds_or_straddles':
                    column_value = column_value.replace("[", "").replace("]", "")
                    dict_game_info['p1_blind_or_straddle'] = column_value.split(',')[0].strip()
                    dict_game_info['p2_blind_or_straddle'] = column_value.split(',')[1].strip()
                    dict_game_info['p3_blind_or_straddle'] = column_value.split(',')[2].strip()
                    dict_game_info['p4_blind_or_straddle'] = column_value.split(',')[3].strip()
                    dict_game_info['p5_blind_or_straddle'] = column_value.split(',')[4].strip()
                    dict_game_info['p6_blind_or_straddle'] = column_value.split(',')[5].strip()
                if column_name == 'starting_stacks':
                    column_value = column_value.replace("[", "").replace("]", "")
                    dict_game_info['p1_starting_stack'] = column_value.split(',')[0].strip()
                    dict_game_info['p2_starting_stack'] = column_value.split(',')[1].strip()
                    dict_game_info['p3_starting_stack'] = column_value.split(',')[2].strip()
                    dict_game_info['p4_starting_stack'] = column_value.split(',')[3].strip()
                    dict_game_info['p5_starting_stack'] = column_value.split(',')[4].strip()
                    dict_game_info['p6_starting_stack'] = column_value.split(',')[5].strip()
                if column_name == 'players':
                    column_value = column_value.replace("[", "").replace("]", "")
                    dict_game_info['p1_player'] = column_value.split(',')[0].strip().replace("'", "")
                    dict_game_info['p2_player'] = column_value.split(',')[1].strip().replace("'", "")
                    dict_game_info['p3_player'] = column_value.split(',')[2].strip().replace("'", "")
                    dict_game_info['p4_player'] = column_value.split(',')[3].strip().replace("'", "")
                    dict_game_info['p5_player'] = column_value.split(',')[4].strip().replace("'", "")
                    dict_game_info['p6_player'] = column_value.split(',')[5].strip().replace("'", "")
                if column_name == 'finishing_stacks':
                    column_value = column_value.replace("[", "").replace("]", "")
                    dict_game_info['p1_finishing_stack'] = column_value.split(',')[0].strip()
                    dict_game_info['p2_finishing_stack'] = column_value.split(',')[1].strip()
                    dict_game_info['p3_finishing_stack'] = column_value.split(',')[2].strip()
                    dict_game_info['p4_finishing_stack'] = column_value.split(',')[3].strip()
                    dict_game_info['p5_finishing_stack'] = column_value.split(',')[4].strip()
                    dict_game_info['p6_finishing_stack'] = column_value.split(',')[5].strip()
                    
                # populate all game actions
                if column_name == 'actions':
                    column_value = column_value.replace("[", "").replace("]", "")
                    actions_array = column_value.split(',')
                    for i in range(0, len(actions_array)):
                        actions_array[i] = actions_array[i].strip().replace("'", "")

                    # populate the sm
                    dict_game_info['p1_sm'] = False
                    dict_game_info['p2_sm'] = False
                    dict_game_info['p3_sm'] = False
                    dict_game_info['p4_sm'] = False
                    dict_game_info['p5_sm'] = False
                    dict_game_info['p6_sm'] = False
                    for action in actions_array:
                        if action.startswith("p1 sm"):
                            dict_game_info['p1_sm'] = True
                        if action.startswith("p2 sm"):
                            dict_game_info['p2_sm'] = True
                        if action.startswith("p3 sm"):
                            dict_game_info['p3_sm'] = True
                        if action.startswith("p4 sm"):
                            dict_game_info['p4_sm'] = True
                        if action.startswith("p5 sm"):
                            dict_game_info['p5_sm'] = True
                        if action.startswith("p6 sm"):
                            dict_game_info['p6_sm'] = True
                            
                    
                    # create the actions_preflop_array
                    actions_preflop_array = []
                    actions_other_1 = []
                    actions_flop_array = []
                    actions_other_2 = []
                    actions_turn_array = []
                    actions_river_array = []
                    actions_preflop_array_2 = []
                    for i in range(0, len(actions_array)):
                        if actions_array[i].startswith("d db "): 
                            actions_preflop_array = actions_array[:i]
                            actions_other_1 = actions_array[i:]
                            break;
                    if len(actions_preflop_array) == 0:
                        actions_preflop_array = actions_array;
                        
                    # create the actions_flop_array
                    if len(actions_other_1) > 1:
                        for i in range(1, len(actions_other_1)):
                            if actions_other_1[i].startswith("d db "): 
                                actions_flop_array = actions_other_1[:i]
                                actions_other_2 = actions_other_1[i:]
                                break;
                        if (len(actions_flop_array) == 0) & (len(actions_other_2) == 0):
                            actions_flop_array = actions_other_1
                    # create the actions_turn_array and the actions_river_array
                    if len(actions_other_2) > 1:
                        for i in range(1, len(actions_other_2)):
                            if actions_other_2[i].startswith("d db "): 
                                actions_turn_array = actions_other_2[:i]
                                actions_river_array = actions_other_2[i:]
                                break;
                        if (len(actions_turn_array) == 0) & (len(actions_river_array) == 0):
                            actions_turn_array = actions_other_2
                    
                    # populate preflop game actions
                    dict_game_actions_preflop['p1_d_dh'] = actions_preflop_array[0].replace("d dh p1 ", "").strip()
                    dict_game_actions_preflop['p2_d_dh'] = actions_preflop_array[1].replace("d dh p2 ", "").strip()
                    dict_game_actions_preflop['p3_d_dh'] = actions_preflop_array[2].replace("d dh p3 ", "").strip()
                    dict_game_actions_preflop['p4_d_dh'] = actions_preflop_array[3].replace("d dh p4 ", "").strip()
                    dict_game_actions_preflop['p5_d_dh'] = actions_preflop_array[4].replace("d dh p5 ", "").strip()
                    dict_game_actions_preflop['p6_d_dh'] = actions_preflop_array[5].replace("d dh p6 ", "").strip()
                    dict_game_actions_preflop['p3_preflop_action_1'] = actions_preflop_array[6].replace("p3 ", "").strip()
                    dict_game_actions_preflop['p4_preflop_action_1'] = actions_preflop_array[7].replace("p4 ", "").strip()
                    dict_game_actions_preflop['p5_preflop_action_1'] = actions_preflop_array[8].replace("p5 ", "").strip()
                    dict_game_actions_preflop['p6_preflop_action_1'] = actions_preflop_array[9].replace("p6 ", "").strip()
                    dict_game_actions_preflop['p1_preflop_action_1'] = actions_preflop_array[10].replace("p1 ", "").strip()
                    if len(actions_preflop_array) > 11:
                        dict_game_actions_preflop['p2_preflop_action_1'] = actions_preflop_array[11].replace("p2 ", "").strip()

                    dict_game_actions_preflop['p3_preflop_action_2'] = ""
                    dict_game_actions_preflop['p4_preflop_action_2'] = ""
                    dict_game_actions_preflop['p5_preflop_action_2'] = ""
                    dict_game_actions_preflop['p6_preflop_action_2'] = ""
                    dict_game_actions_preflop['p1_preflop_action_2'] = ""
                    dict_game_actions_preflop['p2_preflop_action_2'] = ""

                    dict_game_actions_preflop['p3_preflop_action_3'] = ""
                    dict_game_actions_preflop['p4_preflop_action_3'] = ""
                    dict_game_actions_preflop['p5_preflop_action_3'] = ""
                    dict_game_actions_preflop['p6_preflop_action_3'] = ""
                    dict_game_actions_preflop['p1_preflop_action_3'] = ""
                    dict_game_actions_preflop['p2_preflop_action_3'] = ""

                    if len(actions_preflop_array) > 12:
                        actions_preflop_array_2 = actions_preflop_array[12:]
                        action_p1 = 2;
                        action_p2 = 2;
                        action_p3 = 2;
                        action_p4 = 2;
                        action_p5 = 2;
                        action_p6 = 2;
                        for action in actions_preflop_array_2:
                            if (action.startswith("p1 ")) & (not action.startswith("p1 sm")): 
                                dict_game_actions_preflop[f"p1_preflop_action_{action_p1}"] = action.replace("p1 ", "").strip()
                                action_p1 = action_p1 + 1
                            if (action.startswith("p2 ")) & (not action.startswith("p2 sm")): 
                                dict_game_actions_preflop[f"p2_preflop_action_{action_p2}"] = action.replace("p2 ", "").strip()
                                action_p2 = action_p2 + 1
                            if (action.startswith("p3 ")) & (not action.startswith("p3 sm")): 
                                dict_game_actions_preflop[f"p3_preflop_action_{action_p3}"] = action.replace("p3 ", "").strip()
                                action_p3 = action_p3 + 1
                            if (action.startswith("p4 ")) & (not action.startswith("p4 sm")): 
                                dict_game_actions_preflop[f"p4_preflop_action_{action_p4}"] = action.replace("p4 ", "").strip()
                                action_p4 = action_p4 + 1
                            if (action.startswith("p5 ")) & (not action.startswith("p5 sm")): 
                                dict_game_actions_preflop[f"p5_preflop_action_{action_p5}"] = action.replace("p5 ", "").strip()
                                action_p5 = action_p5 + 1
                            if (action.startswith("p6 ")) & (not action.startswith("p6 sm")): 
                                dict_game_actions_preflop[f"p6_preflop_action_{action_p6}"] = action.replace("p6 ", "").strip()
                                action_p6 = action_p6 + 1

                    # populate flop game actions
                    if len(actions_flop_array) > 0:
                        dict_game_actions_flop['flop_d_db'] = actions_flop_array[0].replace("d db ", "").strip()
    
                        dict_game_actions_flop['p3_flop_action_1'] = ""
                        dict_game_actions_flop['p4_flop_action_1'] = ""
                        dict_game_actions_flop['p5_flop_action_1'] = ""
                        dict_game_actions_flop['p6_flop_action_1'] = ""
                        dict_game_actions_flop['p1_flop_action_1'] = ""
                        dict_game_actions_flop['p2_flop_action_1'] = ""
    
                        dict_game_actions_flop['p3_flop_action_2'] = ""
                        dict_game_actions_flop['p4_flop_action_2'] = ""
                        dict_game_actions_flop['p5_flop_action_2'] = ""
                        dict_game_actions_flop['p6_flop_action_2'] = ""
                        dict_game_actions_flop['p1_flop_action_2'] = ""
                        dict_game_actions_flop['p2_flop_action_2'] = ""
    
                        dict_game_actions_flop['p3_flop_action_3'] = ""
                        dict_game_actions_flop['p4_flop_action_3'] = ""
                        dict_game_actions_flop['p5_flop_action_3'] = ""
                        dict_game_actions_flop['p6_flop_action_3'] = ""
                        dict_game_actions_flop['p1_flop_action_3'] = ""
                        dict_game_actions_flop['p2_flop_action_3'] = ""

                    if len(actions_flop_array) > 1:
                        # get the flop_start_player
                        dict_game_actions_flop['flop_start_player'] = actions_flop_array[1][:2]
                        
                        action_p1 = 1;
                        action_p2 = 1;
                        action_p3 = 1;
                        action_p4 = 1;
                        action_p5 = 1;
                        action_p6 = 1;
                        for action in actions_flop_array:
                            if (action.startswith("p1 ")) & (not action.startswith("p1 sm")): 
                                dict_game_actions_flop[f"p1_flop_action_{action_p1}"] = action.replace("p1 ", "").strip()
                                action_p1 = action_p1 + 1
                            if (action.startswith("p2 ")) & (not action.startswith("p2 sm")): 
                                dict_game_actions_flop[f"p2_flop_action_{action_p2}"] = action.replace("p2 ", "").strip()
                                action_p2 = action_p2 + 1
                            if (action.startswith("p3 ")) & (not action.startswith("p3 sm")): 
                                dict_game_actions_flop[f"p3_flop_action_{action_p3}"] = action.replace("p3 ", "").strip()
                                action_p3 = action_p3 + 1
                            if (action.startswith("p4 ")) & (not action.startswith("p4 sm")): 
                                dict_game_actions_flop[f"p4_flop_action_{action_p4}"] = action.replace("p4 ", "").strip()
                                action_p4 = action_p4 + 1
                            if (action.startswith("p5 ")) & (not action.startswith("p5 sm")): 
                                dict_game_actions_flop[f"p5_flop_action_{action_p5}"] = action.replace("p5 ", "").strip()
                                action_p5 = action_p5 + 1
                            if (action.startswith("p6 ")) & (not action.startswith("p6 sm")): 
                                dict_game_actions_flop[f"p6_flop_action_{action_p6}"] = action.replace("p6 ", "").strip()
                                action_p6 = action_p6 + 1

                    # populate turn game actions
                    if len(actions_turn_array) > 0:
                        dict_game_actions_turn['turn_d_db'] = actions_turn_array[0].replace("d db ", "").strip()
    
                        dict_game_actions_turn['p3_turn_action_1'] = ""
                        dict_game_actions_turn['p4_turn_action_1'] = ""
                        dict_game_actions_turn['p5_turn_action_1'] = ""
                        dict_game_actions_turn['p6_turn_action_1'] = ""
                        dict_game_actions_turn['p1_turn_action_1'] = ""
                        dict_game_actions_turn['p2_turn_action_1'] = ""
    
                        dict_game_actions_turn['p3_turn_action_2'] = ""
                        dict_game_actions_turn['p4_turn_action_2'] = ""
                        dict_game_actions_turn['p5_turn_action_2'] = ""
                        dict_game_actions_turn['p6_turn_action_2'] = ""
                        dict_game_actions_turn['p1_turn_action_2'] = ""
                        dict_game_actions_turn['p2_turn_action_2'] = ""
    
                        dict_game_actions_turn['p3_turn_action_3'] = ""
                        dict_game_actions_turn['p4_turn_action_3'] = ""
                        dict_game_actions_turn['p5_turn_action_3'] = ""
                        dict_game_actions_turn['p6_turn_action_3'] = ""
                        dict_game_actions_turn['p1_turn_action_3'] = ""
                        dict_game_actions_turn['p2_turn_action_3'] = ""

                    if len(actions_turn_array) > 1:
                        # get the turn_start_player
                        dict_game_actions_turn['turn_start_player'] = actions_turn_array[1][:2]

                        action_p1 = 1;
                        action_p2 = 1;
                        action_p3 = 1;
                        action_p4 = 1;
                        action_p5 = 1;
                        action_p6 = 1;
                        for action in actions_turn_array:
                            if (action.startswith("p1 ")) & (not action.startswith("p1 sm")): 
                                dict_game_actions_turn[f"p1_turn_action_{action_p1}"] = action.replace("p1 ", "").strip()
                                action_p1 = action_p1 + 1
                            if (action.startswith("p2 ")) & (not action.startswith("p2 sm")): 
                                dict_game_actions_turn[f"p2_turn_action_{action_p2}"] = action.replace("p2 ", "").strip()
                                action_p2 = action_p2 + 1
                            if (action.startswith("p3 ")) & (not action.startswith("p3 sm")): 
                                dict_game_actions_turn[f"p3_turn_action_{action_p3}"] = action.replace("p3 ", "").strip()
                                action_p3 = action_p3 + 1
                            if (action.startswith("p4 ")) & (not action.startswith("p4 sm")): 
                                dict_game_actions_turn[f"p4_turn_action_{action_p4}"] = action.replace("p4 ", "").strip()
                                action_p4 = action_p4 + 1
                            if (action.startswith("p5 ")) & (not action.startswith("p5 sm")): 
                                dict_game_actions_turn[f"p5_turn_action_{action_p5}"] = action.replace("p5 ", "").strip()
                                action_p5 = action_p5 + 1
                            if (action.startswith("p6 ")) & (not action.startswith("p6 sm")): 
                                dict_game_actions_turn[f"p6_turn_action_{action_p6}"] = action.replace("p6 ", "").strip()
                                action_p6 = action_p6 + 1

                    # populate river game actions
                    if len(actions_river_array) > 0:
                        dict_game_actions_river['river_d_db'] = actions_river_array[0].replace("d db ", "").strip()
    
                        dict_game_actions_river['p3_river_action_1'] = ""
                        dict_game_actions_river['p4_river_action_1'] = ""
                        dict_game_actions_river['p5_river_action_1'] = ""
                        dict_game_actions_river['p6_river_action_1'] = ""
                        dict_game_actions_river['p1_river_action_1'] = ""
                        dict_game_actions_river['p2_river_action_1'] = ""
    
                        dict_game_actions_river['p3_river_action_2'] = ""
                        dict_game_actions_river['p4_river_action_2'] = ""
                        dict_game_actions_river['p5_river_action_2'] = ""
                        dict_game_actions_river['p6_river_action_2'] = ""
                        dict_game_actions_river['p1_river_action_2'] = ""
                        dict_game_actions_river['p2_river_action_2'] = ""
    
                        dict_game_actions_river['p3_river_action_3'] = ""
                        dict_game_actions_river['p4_river_action_3'] = ""
                        dict_game_actions_river['p5_river_action_3'] = ""
                        dict_game_actions_river['p6_river_action_3'] = ""
                        dict_game_actions_river['p1_river_action_3'] = ""
                        dict_game_actions_river['p2_river_action_3'] = ""

                    if len(actions_river_array) > 1:
                        # get the river_start_player
                        dict_game_actions_river['river_start_player'] = actions_river_array[1][:2]

                        action_p1 = 1;
                        action_p2 = 1;
                        action_p3 = 1;
                        action_p4 = 1;
                        action_p5 = 1;
                        action_p6 = 1;
                        for action in actions_river_array:
                            if (action.startswith("p1 ")) & (not action.startswith("p1 sm")): 
                                dict_game_actions_river[f"p1_river_action_{action_p1}"] = action.replace("p1 ", "").strip()
                                action_p1 = action_p1 + 1
                            if (action.startswith("p2 ")) & (not action.startswith("p2 sm")): 
                                dict_game_actions_river[f"p2_river_action_{action_p2}"] = action.replace("p2 ", "").strip()
                                action_p2 = action_p2 + 1
                            if (action.startswith("p3 ")) & (not action.startswith("p3 sm")): 
                                dict_game_actions_river[f"p3_river_action_{action_p3}"] = action.replace("p3 ", "").strip()
                                action_p3 = action_p3 + 1
                            if (action.startswith("p4 ")) & (not action.startswith("p4 sm")): 
                                dict_game_actions_river[f"p4_river_action_{action_p4}"] = action.replace("p4 ", "").strip()
                                action_p4 = action_p4 + 1
                            if (action.startswith("p5 ")) & (not action.startswith("p5 sm")): 
                                dict_game_actions_river[f"p5_river_action_{action_p5}"] = action.replace("p5 ", "").strip()
                                action_p5 = action_p5 + 1
                            if (action.startswith("p6 ")) & (not action.startswith("p6 sm")): 
                                dict_game_actions_river[f"p6_river_action_{action_p6}"] = action.replace("p6 ", "").strip()
                                action_p6 = action_p6 + 1
                    
                dict_game_info[column_name] = column_value
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found")

    # Convert dictionaries to DataFrame
    df_game_info = pd.DataFrame(dict_game_info, index=[0])
    df_game_actions_preflop = pd.DataFrame(dict_game_actions_preflop, index=[0])
    df_game_actions_flop = pd.DataFrame(dict_game_actions_flop, index=[0])
    df_game_actions_turn = pd.DataFrame(dict_game_actions_turn, index=[0])
    df_game_actions_river = pd.DataFrame(dict_game_actions_river, index=[0])
    # drop useless columns
    df_game_info = df_game_info.drop(columns=['antes','blinds_or_straddles','starting_stacks', 'actions', 'players', 'finishing_stacks'])
    return df_game_info, df_game_actions_preflop, df_game_actions_flop, df_game_actions_turn, df_game_actions_river

## Function readFolderPHH
This function explores in one folder all the Poker Hand History files representing all the poker games between 6 players and returns 5 dataframes : one about all the games general information, and one for each phase of all the games : preflop, flop, turn and river

In [4]:
def readFolderPHH(folder_path):
    """
    This function explores in one folder all the Poker Hand History files representing all the poker games between 6 players
    and returns 5 dataframes : one about all the games general information, 
    and one for each phase of all the games : preflop, flop, turn and river
    
    Arguments
    ---------
    folder_path: string
           The folder where are located all the PHH files

    Returns
    --------
    df_game_info: pd.DataFrame
         Array representing all the games general information
    df_game_actions_preflop: pd.DataFrame
         Array about the actions played during the preflop phase of all the games
    df_game_actions_flop: pd.DataFrame
         Array about the actions played during the flop phase of all the games
    df_game_actions_turn: pd.DataFrame
         Array about the actions played during the turn phase of all the games
    df_game_actions_river: pd.DataFrame
         Array about the actions played during the river phase of all the games
    """
    df_game_info = pd.DataFrame()
    df_game_actions_preflop = pd.DataFrame()
    df_game_actions_flop = pd.DataFrame()
    df_game_actions_turn = pd.DataFrame()
    df_game_actions_river = pd.DataFrame()
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            if os.path.isfile(file_path):  # Check if the path corresponds to a file
                df_game_info = pd.concat([readFilePHH(file_path)[0], df_game_info], ignore_index=True)
                df_game_actions_preflop = pd.concat([readFilePHH(file_path)[1], df_game_actions_preflop], ignore_index=True)
                df_game_actions_flop = pd.concat([readFilePHH(file_path)[2], df_game_actions_flop], ignore_index=True)
                df_game_actions_turn = pd.concat([readFilePHH(file_path)[3], df_game_actions_turn], ignore_index=True)
                df_game_actions_river = pd.concat([readFilePHH(file_path)[4], df_game_actions_river], ignore_index=True)
    return df_game_info, df_game_actions_preflop, df_game_actions_flop, df_game_actions_turn, df_game_actions_river

## Function getDataframeByGameIDPlayerID
This function reads a dataframe of Poker Games aggregated by Game ID and returns a dataframe aggregated by Game ID and Player ID

In [5]:
def getDataframeByGameIDPlayerID(df_games):
    """
    This function reads a dataframe of Poker Games aggregated by Game ID
    and returns a dataframe aggregated by Game ID and Player ID
    
    Arguments
    ---------
    df_games: pd.DataFrame
           Array aggregated by poker Game ID. There should be one unique row for each Game ID.

    Returns
    --------
    df_games_players: pd.DataFrame
         Array aggregated by poker Game ID and Player ID. There should be one unique row for each combination of Game ID and Player ID.
    """
    df_games_players = pd.DataFrame()
    
    for index, row in df_games.iterrows():
        data_row = {
            'game_id': [row['game_id'], row['game_id'], row['game_id'], row['game_id'], row['game_id'], row['game_id']],
            'small_blind': [row['p1_blind_or_straddle'], row['p1_blind_or_straddle'], row['p1_blind_or_straddle'], row['p1_blind_or_straddle'], row['p1_blind_or_straddle'], row['p1_blind_or_straddle']],
            'big_blind': [row['p2_blind_or_straddle'], row['p2_blind_or_straddle'], row['p2_blind_or_straddle'], row['p2_blind_or_straddle'], row['p2_blind_or_straddle'], row['p2_blind_or_straddle']],
            'min_bet': [row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet']],
            'starting_stack': [row['p1_starting_stack'], row['p2_starting_stack'], row['p3_starting_stack'], row['p4_starting_stack'], row['p5_starting_stack'], row['p6_starting_stack']],
            'player_name': [row['p1_player'], row['p2_player'], row['p3_player'], row['p4_player'], row['p5_player'], row['p6_player']],
            'player_id': ['p1','p2','p3','p4','p5','p6'],
            'sm': [row['p1_sm'], row['p2_sm'], row['p3_sm'], row['p4_sm'], row['p5_sm'], row['p6_sm']],
            'finishing_stack': [row['p1_finishing_stack'], row['p2_finishing_stack'], row['p3_finishing_stack'], row['p4_finishing_stack'], row['p5_finishing_stack'], row['p6_finishing_stack']],
            'd_dh': [row['p1_d_dh'], row['p2_d_dh'], row['p3_d_dh'], row['p4_d_dh'], row['p5_d_dh'], row['p6_d_dh']],
            'preflop_action_1': [row['p1_preflop_action_1'], row['p2_preflop_action_1'], row['p3_preflop_action_1'], row['p4_preflop_action_1'], row['p5_preflop_action_1'], row['p6_preflop_action_1']],
            'preflop_action_2': [row['p1_preflop_action_2'], row['p2_preflop_action_2'], row['p3_preflop_action_2'], row['p4_preflop_action_2'], row['p5_preflop_action_2'], row['p6_preflop_action_2']],
            'preflop_action_3': [row['p1_preflop_action_3'], row['p2_preflop_action_3'], row['p3_preflop_action_3'], row['p4_preflop_action_3'], row['p5_preflop_action_3'], row['p6_preflop_action_3']],
            'flop_d_db': [row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db']],
            'flop_start_player': [row['flop_start_player'], row['flop_start_player'], row['flop_start_player'], row['flop_start_player'], row['flop_start_player'], row['flop_start_player']],
            'flop_action_1': [row['p1_flop_action_1'], row['p2_flop_action_1'], row['p3_flop_action_1'], row['p4_flop_action_1'], row['p5_flop_action_1'], row['p6_flop_action_1']],
            'flop_action_2': [row['p1_flop_action_2'], row['p2_flop_action_2'], row['p3_flop_action_2'], row['p4_flop_action_2'], row['p5_flop_action_2'], row['p6_flop_action_2']],
            'flop_action_3': [row['p1_flop_action_3'], row['p2_flop_action_3'], row['p3_flop_action_3'], row['p4_flop_action_3'], row['p5_flop_action_3'], row['p6_flop_action_3']],
            'turn_d_db': [row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db']],
            'turn_start_player': [row['turn_start_player'], row['turn_start_player'], row['turn_start_player'], row['turn_start_player'], row['turn_start_player'], row['turn_start_player']],
            'turn_action_1': [row['p1_turn_action_1'], row['p2_turn_action_1'], row['p3_turn_action_1'], row['p4_turn_action_1'], row['p5_turn_action_1'], row['p6_turn_action_1']],
            'turn_action_2': [row['p1_turn_action_2'], row['p2_turn_action_2'], row['p3_turn_action_2'], row['p4_turn_action_2'], row['p5_turn_action_2'], row['p6_turn_action_2']],
            'turn_action_3': [row['p1_turn_action_3'], row['p2_turn_action_3'], row['p3_turn_action_3'], row['p4_turn_action_3'], row['p5_turn_action_3'], row['p6_turn_action_3']],
            'river_d_db': [row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db']],
            'river_start_player': [row['river_start_player'], row['river_start_player'], row['river_start_player'], row['river_start_player'], row['river_start_player'], row['river_start_player']],
            'river_action_1': [row['p1_river_action_1'], row['p2_river_action_1'], row['p3_river_action_1'], row['p4_river_action_1'], row['p5_river_action_1'], row['p6_river_action_1']],
            'river_action_2': [row['p1_river_action_2'], row['p2_river_action_2'], row['p3_river_action_2'], row['p4_river_action_2'], row['p5_river_action_2'], row['p6_river_action_2']],
            'river_action_3': [row['p1_river_action_3'], row['p2_river_action_3'], row['p3_river_action_3'], row['p4_river_action_3'], row['p5_river_action_3'], row['p6_river_action_3']],
        }
        df_games_players = pd.concat([pd.DataFrame(data_row), df_games_players], ignore_index=True)
    return df_games_players

## Function getDataframeByGameIDTurnID
This function reads a dataframe of Poker Games aggregated by Game ID and Player ID and returns a dataframe aggregated by Game ID and Turn ID

In [6]:
def getDataframeByGameIDTurnID(df_games_players):
    """
    This function reads a dataframe of Poker Games aggregated by Game ID and Player ID
    and returns a dataframe aggregated by Game ID and Turn ID
    
    Arguments
    ---------
    df_games_players: pd.DataFrame
           Array aggregated by poker Game ID and Player ID. There should be one unique row for each combination of Game ID and Player ID

    Returns
    --------
    df_games_turns: pd.DataFrame
         Array aggregated by poker Game ID and Turn ID. There should be one unique row for each combination of Game ID, Player ID and Turn ID.
    """
    df_games_turns = pd.DataFrame()
    
    for index, row in df_games_players.iterrows():
        data_row = {
            'game_id': [row['game_id'], row['game_id'], row['game_id'], row['game_id'], row['game_id'], row['game_id'],row['game_id'], row['game_id'], row['game_id'], row['game_id'], row['game_id'], row['game_id']],
            'small_blind': [row['small_blind'], row['small_blind'], row['small_blind'], row['small_blind'], row['small_blind'], row['small_blind'],row['small_blind'], row['small_blind'], row['small_blind'], row['small_blind'], row['small_blind'], row['small_blind']],
            'big_blind': [row['big_blind'], row['big_blind'], row['big_blind'], row['big_blind'], row['big_blind'], row['big_blind'],row['big_blind'], row['big_blind'], row['big_blind'], row['big_blind'], row['big_blind'], row['big_blind']],
            'min_bet': [row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet'],row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet'], row['min_bet']],
            'starting_stack': [row['starting_stack'], row['starting_stack'], row['starting_stack'], row['starting_stack'], row['starting_stack'], row['starting_stack'],row['starting_stack'], row['starting_stack'], row['starting_stack'], row['starting_stack'], row['starting_stack'], row['starting_stack']],
            'player_name': [row['player_name'], row['player_name'], row['player_name'], row['player_name'], row['player_name'], row['player_name'],row['player_name'], row['player_name'], row['player_name'], row['player_name'], row['player_name'], row['player_name']],
            'player_id': [row['player_id'], row['player_id'], row['player_id'], row['player_id'], row['player_id'], row['player_id'],row['player_id'], row['player_id'], row['player_id'], row['player_id'], row['player_id'], row['player_id']],
            'sm': [row['sm'], row['sm'], row['sm'], row['sm'], row['sm'], row['sm'],row['sm'], row['sm'], row['sm'], row['sm'], row['sm'], row['sm']],
            'finishing_stack': [row['finishing_stack'], row['finishing_stack'], row['finishing_stack'], row['finishing_stack'], row['finishing_stack'], row['finishing_stack'],row['finishing_stack'], row['finishing_stack'], row['finishing_stack'], row['finishing_stack'], row['finishing_stack'], row['finishing_stack']],
            'd_dh': [row['d_dh'], row['d_dh'], row['d_dh'], row['d_dh'], row['d_dh'], row['d_dh'],row['d_dh'], row['d_dh'], row['d_dh'], row['d_dh'], row['d_dh'], row['d_dh']],
            'flop_d_db': [row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db'],row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db'], row['flop_d_db']],
            'turn_d_db': [row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db'],row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db'], row['turn_d_db']],
            'river_d_db': [row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db'],row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db'], row['river_d_db']],
            'stage': ['preflop', 'preflop', 'preflop', 'flop', 'flop', 'flop', 'turn', 'turn', 'turn', 'river', 'river', 'river'],
            'sub_stage': ['preflop_1', 'preflop_2', 'preflop_3', 'flop_1', 'flop_2', 'flop_3', 'turn_1', 'turn_2', 'turn_3', 'river_1', 'river_2', 'river_3'],
            'turn_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            'action': [row['preflop_action_1'], row['preflop_action_2'], row['preflop_action_3'], row['flop_action_1'], row['flop_action_2'], row['flop_action_3'],row['turn_action_1'], row['turn_action_2'], row['turn_action_3'], row['river_action_1'], row['river_action_2'], row['river_action_3']],
            'action_raise_amount': [row['preflop_action_1_cbr_delta'], row['preflop_action_2_cbr_delta'], row['preflop_action_3_cbr_delta'], row['flop_action_1_cbr_delta'], row['flop_action_2_cbr_delta'], row['flop_action_3_cbr_delta'],row['turn_action_1_cbr_delta'], row['turn_action_2_cbr_delta'], row['turn_action_3_cbr_delta'], row['river_action_1_cbr_delta'], row['river_action_2_cbr_delta'], row['river_action_3_cbr_delta']],
            'nb_previous_raise': [row['preflop_action_1_nb_previous_raise'], row['preflop_action_2_nb_previous_raise'], row['preflop_action_3_nb_previous_raise'], row['flop_action_1_nb_previous_raise'], row['flop_action_2_nb_previous_raise'], row['flop_action_3_nb_previous_raise'],row['turn_action_1_nb_previous_raise'], row['turn_action_2_nb_previous_raise'], row['turn_action_3_nb_previous_raise'], row['river_action_1_nb_previous_raise'], row['river_action_2_nb_previous_raise'], row['river_action_3_nb_previous_raise']],
            'nb_players_left': [row['preflop_action_1_nb_players_left'], row['preflop_action_2_nb_players_left'], row['preflop_action_3_nb_players_left'], row['flop_action_1_nb_players_left'], row['flop_action_2_nb_players_left'], row['flop_action_3_nb_players_left'],row['turn_action_1_nb_players_left'], row['turn_action_2_nb_players_left'], row['turn_action_3_nb_players_left'], row['river_action_1_nb_players_left'], row['river_action_2_nb_players_left'], row['river_action_3_nb_players_left']],
            'amount_already_in': [row['preflop_action_1_cbr_already_in'], row['preflop_action_2_cbr_already_in'], row['preflop_action_3_cbr_already_in'], row['flop_action_1_cbr_already_in'], row['flop_action_2_cbr_already_in'], row['flop_action_3_cbr_already_in'],row['turn_action_1_cbr_already_in'], row['turn_action_2_cbr_already_in'], row['turn_action_3_cbr_already_in'], row['river_action_1_cbr_already_in'], row['river_action_2_cbr_already_in'], row['river_action_3_cbr_already_in']],
            'amount_to_continue': [row['preflop_action_1_cbr_to_continue'], row['preflop_action_2_cbr_to_continue'], row['preflop_action_3_cbr_to_continue'], row['flop_action_1_cbr_to_continue'], row['flop_action_2_cbr_to_continue'], row['flop_action_3_cbr_to_continue'],row['turn_action_1_cbr_to_continue'], row['turn_action_2_cbr_to_continue'], row['turn_action_3_cbr_to_continue'], row['river_action_1_cbr_to_continue'], row['river_action_2_cbr_to_continue'], row['river_action_3_cbr_to_continue']],
        }
        df_games_turns = pd.concat([pd.DataFrame(data_row), df_games_turns], ignore_index=True)
        
        # remove all rows where action is empty as it means no actions were performed during that turn and it is not relevant
        df_games_turns.loc[df_games_turns['action'] == '', 'action'] = np.nan
        df_games_turns = df_games_turns.dropna(subset=['action'])
        # keep only the action code in column action and remove the amount raised
        df_games_turns['action'] = df_games_turns['action'].str[:3]
        # sort the dataframe by game id, player id, turn number
        df_games_turns = df_games_turns.sort_values(
            by=['game_id', 'player_id', 'turn_id'],
            ascending=[True, True, True]
        )
    return df_games_turns

## Function enrichPlayersDataframe
This function reads a dataframe of Poker Games aggregated by Game ID and Player ID and enrich it with multiple additional columns

In [7]:
def enrichPlayersDataframe(df_games_players):
    """
    This function reads a dataframe of Poker Games aggregated by Game ID and Player ID
    and enrich it with multiple additional columns
    
    Arguments
    ---------
    df_games_players: pd.DataFrame
           Array aggregated by poker Game ID and Player ID. There should be one unique row for each combination of Game ID and Player ID

    Returns
    --------
    df_games_players_enriched: pd.DataFrame
         Array aggregated by poker Game ID and Player ID. It was enriched with multiple additional columns
    """
    # For i from 1 to 6 :
    # We add columns pi_player_id, pi_preflop_action_1, pi_preflop_action_2, pi_preflop_action_3, pi_flop_action_1, pi_flop_action_2, pi_flop_action_3,
    # pi_turn_action_1, pi_turn_action_2, pi_turn_action_3, pi_river_action_1, pi_river_action_2, pi_river_action_3
    df_p1 = df_games_players[df_games_players['player_id'] == 'p1']
    df_p2 = df_games_players[df_games_players['player_id'] == 'p2']
    df_p3 = df_games_players[df_games_players['player_id'] == 'p3']
    df_p4 = df_games_players[df_games_players['player_id'] == 'p4']
    df_p5 = df_games_players[df_games_players['player_id'] == 'p5']
    df_p6 = df_games_players[df_games_players['player_id'] == 'p6']
    
    df_p1 = df_p1.drop(columns=['small_blind', 'big_blind', 'min_bet', 'starting_stack', 'player_name', 'sm', 'finishing_stack', 'd_dh', 'flop_d_db', 'flop_start_player', 'turn_d_db', 'turn_start_player', 'river_d_db', 'river_start_player'])
    df_p1 = df_p1.rename(columns = {'player_id':'p1_player_id', 'preflop_action_1':'p1_preflop_action_1', 'preflop_action_2':'p1_preflop_action_2', 'preflop_action_3':'p1_preflop_action_3'})
    df_p1 = df_p1.rename(columns = {'flop_action_1':'p1_flop_action_1', 'flop_action_2':'p1_flop_action_2', 'flop_action_3':'p1_flop_action_3'})
    df_p1 = df_p1.rename(columns = {'turn_action_1':'p1_turn_action_1', 'turn_action_2':'p1_turn_action_2', 'turn_action_3':'p1_turn_action_3'})
    df_p1 = df_p1.rename(columns = {'river_action_1':'p1_river_action_1', 'river_action_2':'p1_river_action_2', 'river_action_3':'p1_river_action_3'})
    
    df_p2 = df_p2.drop(columns=['small_blind', 'big_blind', 'min_bet', 'starting_stack', 'player_name', 'sm', 'finishing_stack', 'd_dh', 'flop_d_db', 'flop_start_player', 'turn_d_db', 'turn_start_player', 'river_d_db', 'river_start_player'])
    df_p2 = df_p2.rename(columns = {'player_id':'p2_player_id', 'preflop_action_1':'p2_preflop_action_1', 'preflop_action_2':'p2_preflop_action_2', 'preflop_action_3':'p2_preflop_action_3'})
    df_p2 = df_p2.rename(columns = {'flop_action_1':'p2_flop_action_1', 'flop_action_2':'p2_flop_action_2', 'flop_action_3':'p2_flop_action_3'})
    df_p2 = df_p2.rename(columns = {'turn_action_1':'p2_turn_action_1', 'turn_action_2':'p2_turn_action_2', 'turn_action_3':'p2_turn_action_3'})
    df_p2 = df_p2.rename(columns = {'river_action_1':'p2_river_action_1', 'river_action_2':'p2_river_action_2', 'river_action_3':'p2_river_action_3'})
    
    df_p3 = df_p3.drop(columns=['small_blind', 'big_blind', 'min_bet', 'starting_stack', 'player_name', 'sm', 'finishing_stack', 'd_dh', 'flop_d_db', 'flop_start_player', 'turn_d_db', 'turn_start_player', 'river_d_db', 'river_start_player'])
    df_p3 = df_p3.rename(columns = {'player_id':'p3_player_id', 'preflop_action_1':'p3_preflop_action_1', 'preflop_action_2':'p3_preflop_action_2', 'preflop_action_3':'p3_preflop_action_3'})
    df_p3 = df_p3.rename(columns = {'flop_action_1':'p3_flop_action_1', 'flop_action_2':'p3_flop_action_2', 'flop_action_3':'p3_flop_action_3'})
    df_p3 = df_p3.rename(columns = {'turn_action_1':'p3_turn_action_1', 'turn_action_2':'p3_turn_action_2', 'turn_action_3':'p3_turn_action_3'})
    df_p3 = df_p3.rename(columns = {'river_action_1':'p3_river_action_1', 'river_action_2':'p3_river_action_2', 'river_action_3':'p3_river_action_3'})
    
    df_p4 = df_p4.drop(columns=['small_blind', 'big_blind', 'min_bet', 'starting_stack', 'player_name', 'sm', 'finishing_stack', 'd_dh', 'flop_d_db', 'flop_start_player', 'turn_d_db', 'turn_start_player', 'river_d_db', 'river_start_player'])
    df_p4 = df_p4.rename(columns = {'player_id':'p4_player_id', 'preflop_action_1':'p4_preflop_action_1', 'preflop_action_2':'p4_preflop_action_2', 'preflop_action_3':'p4_preflop_action_3'})
    df_p4 = df_p4.rename(columns = {'flop_action_1':'p4_flop_action_1', 'flop_action_2':'p4_flop_action_2', 'flop_action_3':'p4_flop_action_3'})
    df_p4 = df_p4.rename(columns = {'turn_action_1':'p4_turn_action_1', 'turn_action_2':'p4_turn_action_2', 'turn_action_3':'p4_turn_action_3'})
    df_p4 = df_p4.rename(columns = {'river_action_1':'p4_river_action_1', 'river_action_2':'p4_river_action_2', 'river_action_3':'p4_river_action_3'})
    
    df_p5 = df_p5.drop(columns=['small_blind', 'big_blind', 'min_bet', 'starting_stack', 'player_name', 'sm', 'finishing_stack', 'd_dh', 'flop_d_db', 'flop_start_player', 'turn_d_db', 'turn_start_player', 'river_d_db', 'river_start_player'])
    df_p5 = df_p5.rename(columns = {'player_id':'p5_player_id', 'preflop_action_1':'p5_preflop_action_1', 'preflop_action_2':'p5_preflop_action_2', 'preflop_action_3':'p5_preflop_action_3'})
    df_p5 = df_p5.rename(columns = {'flop_action_1':'p5_flop_action_1', 'flop_action_2':'p5_flop_action_2', 'flop_action_3':'p5_flop_action_3'})
    df_p5 = df_p5.rename(columns = {'turn_action_1':'p5_turn_action_1', 'turn_action_2':'p5_turn_action_2', 'turn_action_3':'p5_turn_action_3'})
    df_p5 = df_p5.rename(columns = {'river_action_1':'p5_river_action_1', 'river_action_2':'p5_river_action_2', 'river_action_3':'p5_river_action_3'})
    
    df_p6 = df_p6.drop(columns=['small_blind', 'big_blind', 'min_bet', 'starting_stack', 'player_name', 'sm', 'finishing_stack', 'd_dh', 'flop_d_db', 'flop_start_player', 'turn_d_db', 'turn_start_player', 'river_d_db', 'river_start_player'])
    df_p6 = df_p6.rename(columns = {'player_id':'p6_player_id', 'preflop_action_1':'p6_preflop_action_1', 'preflop_action_2':'p6_preflop_action_2', 'preflop_action_3':'p6_preflop_action_3'})
    df_p6 = df_p6.rename(columns = {'flop_action_1':'p6_flop_action_1', 'flop_action_2':'p6_flop_action_2', 'flop_action_3':'p6_flop_action_3'})
    df_p6 = df_p6.rename(columns = {'turn_action_1':'p6_turn_action_1', 'turn_action_2':'p6_turn_action_2', 'turn_action_3':'p6_turn_action_3'})
    df_p6 = df_p6.rename(columns = {'river_action_1':'p6_river_action_1', 'river_action_2':'p6_river_action_2', 'river_action_3':'p6_river_action_3'})

    df_temp_1 = pd.merge(left = df_games_players, right = df_p1, left_on = 'game_id', right_on = 'game_id', how = 'inner')
    df_temp_2 = pd.merge(left = df_temp_1, right = df_p2, left_on = 'game_id', right_on = 'game_id', how = 'inner')
    df_temp_3 = pd.merge(left = df_temp_2, right = df_p3, left_on = 'game_id', right_on = 'game_id', how = 'inner')
    df_temp_4 = pd.merge(left = df_temp_3, right = df_p4, left_on = 'game_id', right_on = 'game_id', how = 'inner')
    df_temp_5 = pd.merge(left = df_temp_4, right = df_p5, left_on = 'game_id', right_on = 'game_id', how = 'inner')
    df_games_players_enriched = pd.merge(left = df_temp_5, right = df_p6, left_on = 'game_id', right_on = 'game_id', how = 'inner')

    # add column preflop_action_1_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_1_nb_players_left'] = 6
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_1_nb_players_left'] = 6 - df_games_players_enriched['p3_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_1_nb_players_left'] = 6 - df_games_players_enriched['p3_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_1_nb_players_left'] = 6 - df_games_players_enriched['p3_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_1_nb_players_left'] = 6 - df_games_players_enriched['p3_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_1_nb_players_left'] = 6 - df_games_players_enriched['p3_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column preflop_action_2_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_2_nb_players_left'] = df_games_players_enriched['preflop_action_1_nb_players_left'] - df_games_players_enriched['p3_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_2_nb_players_left'] = df_games_players_enriched['preflop_action_1_nb_players_left'] - df_games_players_enriched['p4_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_2_nb_players_left'] = df_games_players_enriched['preflop_action_1_nb_players_left'] - df_games_players_enriched['p5_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_2_nb_players_left'] = df_games_players_enriched['preflop_action_1_nb_players_left'] - df_games_players_enriched['p6_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_2_nb_players_left'] = df_games_players_enriched['preflop_action_1_nb_players_left'] - df_games_players_enriched['p1_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_2_nb_players_left'] = df_games_players_enriched['preflop_action_1_nb_players_left'] - df_games_players_enriched['p2_preflop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column preflop_action_3_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_3_nb_players_left'] = df_games_players_enriched['preflop_action_2_nb_players_left'] - df_games_players_enriched['p3_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_3_nb_players_left'] = df_games_players_enriched['preflop_action_2_nb_players_left'] - df_games_players_enriched['p4_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_3_nb_players_left'] = df_games_players_enriched['preflop_action_2_nb_players_left'] - df_games_players_enriched['p5_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_3_nb_players_left'] = df_games_players_enriched['preflop_action_2_nb_players_left'] - df_games_players_enriched['p6_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_3_nb_players_left'] = df_games_players_enriched['preflop_action_2_nb_players_left'] - df_games_players_enriched['p1_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_3_nb_players_left'] = df_games_players_enriched['preflop_action_2_nb_players_left'] - df_games_players_enriched['p2_preflop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column flop_action_1_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_1_nb_players_left'] = df_games_players_enriched['preflop_action_3_nb_players_left'] - df_games_players_enriched['p1_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_1_nb_players_left'] = df_games_players_enriched['preflop_action_3_nb_players_left'] - df_games_players_enriched['p2_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_1_nb_players_left'] = df_games_players_enriched['preflop_action_3_nb_players_left'] - df_games_players_enriched['p1_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_1_nb_players_left'] = df_games_players_enriched['preflop_action_3_nb_players_left'] - df_games_players_enriched['p1_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_1_nb_players_left'] = df_games_players_enriched['preflop_action_3_nb_players_left'] - df_games_players_enriched['p1_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_1_nb_players_left'] = df_games_players_enriched['preflop_action_3_nb_players_left'] - df_games_players_enriched['p1_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_preflop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column flop_action_2_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_2_nb_players_left'] = df_games_players_enriched['flop_action_1_nb_players_left'] - df_games_players_enriched['p1_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_2_nb_players_left'] = df_games_players_enriched['flop_action_1_nb_players_left'] - df_games_players_enriched['p2_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_2_nb_players_left'] = df_games_players_enriched['flop_action_1_nb_players_left'] - df_games_players_enriched['p3_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_2_nb_players_left'] = df_games_players_enriched['flop_action_1_nb_players_left'] - df_games_players_enriched['p4_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_2_nb_players_left'] = df_games_players_enriched['flop_action_1_nb_players_left'] - df_games_players_enriched['p5_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_2_nb_players_left'] = df_games_players_enriched['flop_action_1_nb_players_left'] - df_games_players_enriched['p6_flop_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column flop_action_3_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_3_nb_players_left'] = df_games_players_enriched['flop_action_2_nb_players_left'] - df_games_players_enriched['p1_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_3_nb_players_left'] = df_games_players_enriched['flop_action_2_nb_players_left'] - df_games_players_enriched['p2_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_3_nb_players_left'] = df_games_players_enriched['flop_action_2_nb_players_left'] - df_games_players_enriched['p3_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_3_nb_players_left'] = df_games_players_enriched['flop_action_2_nb_players_left'] - df_games_players_enriched['p4_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_3_nb_players_left'] = df_games_players_enriched['flop_action_2_nb_players_left'] - df_games_players_enriched['p5_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_3_nb_players_left'] = df_games_players_enriched['flop_action_2_nb_players_left'] - df_games_players_enriched['p6_flop_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column turn_action_1_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_1_nb_players_left'] = df_games_players_enriched['flop_action_3_nb_players_left'] - df_games_players_enriched['p1_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_1_nb_players_left'] = df_games_players_enriched['flop_action_3_nb_players_left'] - df_games_players_enriched['p2_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_1_nb_players_left'] = df_games_players_enriched['flop_action_3_nb_players_left'] - df_games_players_enriched['p3_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_1_nb_players_left'] = df_games_players_enriched['flop_action_3_nb_players_left'] - df_games_players_enriched['p4_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_1_nb_players_left'] = df_games_players_enriched['flop_action_3_nb_players_left'] - df_games_players_enriched['p5_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_1_nb_players_left'] = df_games_players_enriched['flop_action_3_nb_players_left'] - df_games_players_enriched['p6_flop_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column turn_action_2_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_2_nb_players_left'] = df_games_players_enriched['turn_action_1_nb_players_left'] - df_games_players_enriched['p1_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_2_nb_players_left'] = df_games_players_enriched['turn_action_1_nb_players_left'] - df_games_players_enriched['p2_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_2_nb_players_left'] = df_games_players_enriched['turn_action_1_nb_players_left'] - df_games_players_enriched['p3_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_2_nb_players_left'] = df_games_players_enriched['turn_action_1_nb_players_left'] - df_games_players_enriched['p4_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_2_nb_players_left'] = df_games_players_enriched['turn_action_1_nb_players_left'] - df_games_players_enriched['p5_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_2_nb_players_left'] = df_games_players_enriched['turn_action_1_nb_players_left'] - df_games_players_enriched['p6_turn_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column turn_action_3_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_3_nb_players_left'] = df_games_players_enriched['turn_action_2_nb_players_left'] - df_games_players_enriched['p1_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_3_nb_players_left'] = df_games_players_enriched['turn_action_2_nb_players_left'] - df_games_players_enriched['p2_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_3_nb_players_left'] = df_games_players_enriched['turn_action_2_nb_players_left'] - df_games_players_enriched['p3_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_3_nb_players_left'] = df_games_players_enriched['turn_action_2_nb_players_left'] - df_games_players_enriched['p4_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_3_nb_players_left'] = df_games_players_enriched['turn_action_2_nb_players_left'] - df_games_players_enriched['p5_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_3_nb_players_left'] = df_games_players_enriched['turn_action_2_nb_players_left'] - df_games_players_enriched['p6_turn_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column river_action_1_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_1_nb_players_left'] = df_games_players_enriched['turn_action_3_nb_players_left'] - df_games_players_enriched['p1_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_1_nb_players_left'] = df_games_players_enriched['turn_action_3_nb_players_left'] - df_games_players_enriched['p2_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_1_nb_players_left'] = df_games_players_enriched['turn_action_3_nb_players_left'] - df_games_players_enriched['p3_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_1_nb_players_left'] = df_games_players_enriched['turn_action_3_nb_players_left'] - df_games_players_enriched['p4_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_1_nb_players_left'] = df_games_players_enriched['turn_action_3_nb_players_left'] - df_games_players_enriched['p5_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_1_nb_players_left'] = df_games_players_enriched['turn_action_3_nb_players_left'] - df_games_players_enriched['p6_turn_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column river_action_2_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_2_nb_players_left'] = df_games_players_enriched['river_action_1_nb_players_left'] - df_games_players_enriched['p1_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_1'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_2_nb_players_left'] = df_games_players_enriched['river_action_1_nb_players_left'] - df_games_players_enriched['p2_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_2_nb_players_left'] = df_games_players_enriched['river_action_1_nb_players_left'] - df_games_players_enriched['p3_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_2_nb_players_left'] = df_games_players_enriched['river_action_1_nb_players_left'] - df_games_players_enriched['p4_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_2_nb_players_left'] = df_games_players_enriched['river_action_1_nb_players_left'] - df_games_players_enriched['p5_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_2_nb_players_left'] = df_games_players_enriched['river_action_1_nb_players_left'] - df_games_players_enriched['p6_river_action_1'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    
    # add column river_action_3_nb_players_left
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_3_nb_players_left'] = df_games_players_enriched['river_action_2_nb_players_left'] - df_games_players_enriched['p1_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_2'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_3_nb_players_left'] = df_games_players_enriched['river_action_2_nb_players_left'] - df_games_players_enriched['p2_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_3_nb_players_left'] = df_games_players_enriched['river_action_2_nb_players_left'] - df_games_players_enriched['p3_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_3_nb_players_left'] = df_games_players_enriched['river_action_2_nb_players_left'] - df_games_players_enriched['p4_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_3_nb_players_left'] = df_games_players_enriched['river_action_2_nb_players_left'] - df_games_players_enriched['p5_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p6_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_3'].apply(lambda x: 1 if x == 'f' else 0)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_3_nb_players_left'] = df_games_players_enriched['river_action_2_nb_players_left'] - df_games_players_enriched['p6_river_action_2'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p1_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p2_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p3_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p4_river_action_3'].apply(lambda x: 1 if x == 'f' else 0) - df_games_players_enriched['p5_river_action_3'].apply(lambda x: 1 if x == 'f' else 0)

    # add columns for cbr_values
    df_games_players_enriched['preflop_action_1_cbr'] = df_games_players_enriched['preflop_action_1'].str[4:]
    df_games_players_enriched['preflop_action_1_cbr'] = df_games_players_enriched['preflop_action_1_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['preflop_action_2_cbr'] = df_games_players_enriched['preflop_action_2'].str[4:]
    df_games_players_enriched['preflop_action_2_cbr'] = df_games_players_enriched['preflop_action_2_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['preflop_action_3_cbr'] = df_games_players_enriched['preflop_action_3'].str[4:]
    df_games_players_enriched['preflop_action_3_cbr'] = df_games_players_enriched['preflop_action_3_cbr'].replace('', 0).fillna(0)
    
    df_games_players_enriched['flop_action_1_cbr'] = df_games_players_enriched['flop_action_1'].str[4:]
    df_games_players_enriched['flop_action_1_cbr'] = df_games_players_enriched['flop_action_1_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['flop_action_2_cbr'] = df_games_players_enriched['flop_action_2'].str[4:]
    df_games_players_enriched['flop_action_2_cbr'] = df_games_players_enriched['flop_action_2_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['flop_action_3_cbr'] = df_games_players_enriched['flop_action_3'].str[4:]
    df_games_players_enriched['flop_action_3_cbr'] = df_games_players_enriched['flop_action_3_cbr'].replace('', 0).fillna(0)
    
    df_games_players_enriched['turn_action_1_cbr'] = df_games_players_enriched['turn_action_1'].str[4:]
    df_games_players_enriched['turn_action_1_cbr'] = df_games_players_enriched['turn_action_1_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['turn_action_2_cbr'] = df_games_players_enriched['turn_action_2'].str[4:]
    df_games_players_enriched['turn_action_2_cbr'] = df_games_players_enriched['turn_action_2_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['turn_action_3_cbr'] = df_games_players_enriched['turn_action_3'].str[4:]
    df_games_players_enriched['turn_action_3_cbr'] = df_games_players_enriched['turn_action_3_cbr'].replace('', 0).fillna(0)
    
    df_games_players_enriched['river_action_1_cbr'] = df_games_players_enriched['river_action_1'].str[4:]
    df_games_players_enriched['river_action_1_cbr'] = df_games_players_enriched['river_action_1_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['river_action_2_cbr'] = df_games_players_enriched['river_action_2'].str[4:]
    df_games_players_enriched['river_action_2_cbr'] = df_games_players_enriched['river_action_2_cbr'].replace('', 0).fillna(0)
    df_games_players_enriched['river_action_3_cbr'] = df_games_players_enriched['river_action_3'].str[4:]
    df_games_players_enriched['river_action_3_cbr'] = df_games_players_enriched['river_action_3_cbr'].replace('', 0).fillna(0)

    # add columns for number of times the player has already raised before
    df_games_players_enriched['preflop_action_1_nb_previous_raise'] = 0
    df_games_players_enriched['preflop_action_2_nb_previous_raise'] = df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    df_games_players_enriched['preflop_action_3_nb_previous_raise'] = df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    
    df_games_players_enriched['flop_action_1_nb_previous_raise'] = df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    df_games_players_enriched['flop_action_2_nb_previous_raise'] = df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    df_games_players_enriched['flop_action_3_nb_previous_raise'] = df_games_players_enriched['flop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    
    df_games_players_enriched['turn_action_1_nb_previous_raise'] = df_games_players_enriched['flop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    df_games_players_enriched['turn_action_2_nb_previous_raise'] = df_games_players_enriched['turn_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    df_games_players_enriched['turn_action_3_nb_previous_raise'] = df_games_players_enriched['turn_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    
    df_games_players_enriched['river_action_1_nb_previous_raise'] = df_games_players_enriched['turn_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    df_games_players_enriched['river_action_2_nb_previous_raise'] = df_games_players_enriched['river_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)
    df_games_players_enriched['river_action_3_nb_previous_raise'] = df_games_players_enriched['river_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['river_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['turn_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['flop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_3_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_2_cbr'].apply(lambda x: 1 if x != 0 else 0) + df_games_players_enriched['preflop_action_1_cbr'].apply(lambda x: 1 if x != 0 else 0)

    # add columns preflop_action_1_cbr_pi
    df_temp_preflop_action_1_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'preflop_action_1_cbr']]
    df_temp_preflop_action_1_cbr_p1 = df_temp_preflop_action_1_cbr_p1.rename(columns = {'preflop_action_1_cbr':'preflop_action_1_cbr_p1'})
    df_temp_preflop_action_1_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'preflop_action_1_cbr']]
    df_temp_preflop_action_1_cbr_p2 = df_temp_preflop_action_1_cbr_p2.rename(columns = {'preflop_action_1_cbr':'preflop_action_1_cbr_p2'})
    df_temp_preflop_action_1_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'preflop_action_1_cbr']]
    df_temp_preflop_action_1_cbr_p3 = df_temp_preflop_action_1_cbr_p3.rename(columns = {'preflop_action_1_cbr':'preflop_action_1_cbr_p3'})
    df_temp_preflop_action_1_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'preflop_action_1_cbr']]
    df_temp_preflop_action_1_cbr_p4 = df_temp_preflop_action_1_cbr_p4.rename(columns = {'preflop_action_1_cbr':'preflop_action_1_cbr_p4'})
    df_temp_preflop_action_1_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'preflop_action_1_cbr']]
    df_temp_preflop_action_1_cbr_p5 = df_temp_preflop_action_1_cbr_p5.rename(columns = {'preflop_action_1_cbr':'preflop_action_1_cbr_p5'})
    df_temp_preflop_action_1_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'preflop_action_1_cbr']]
    df_temp_preflop_action_1_cbr_p6 = df_temp_preflop_action_1_cbr_p6.rename(columns = {'preflop_action_1_cbr':'preflop_action_1_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_preflop_action_1_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_preflop_action_1_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_preflop_action_1_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_preflop_action_1_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_preflop_action_1_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_preflop_action_1_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns preflop_action_2_cbr_pi
    df_temp_preflop_action_2_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'preflop_action_2_cbr']]
    df_temp_preflop_action_2_cbr_p1 = df_temp_preflop_action_2_cbr_p1.rename(columns = {'preflop_action_2_cbr':'preflop_action_2_cbr_p1'})
    df_temp_preflop_action_2_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'preflop_action_2_cbr']]
    df_temp_preflop_action_2_cbr_p2 = df_temp_preflop_action_2_cbr_p2.rename(columns = {'preflop_action_2_cbr':'preflop_action_2_cbr_p2'})
    df_temp_preflop_action_2_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'preflop_action_2_cbr']]
    df_temp_preflop_action_2_cbr_p3 = df_temp_preflop_action_2_cbr_p3.rename(columns = {'preflop_action_2_cbr':'preflop_action_2_cbr_p3'})
    df_temp_preflop_action_2_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'preflop_action_2_cbr']]
    df_temp_preflop_action_2_cbr_p4 = df_temp_preflop_action_2_cbr_p4.rename(columns = {'preflop_action_2_cbr':'preflop_action_2_cbr_p4'})
    df_temp_preflop_action_2_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'preflop_action_2_cbr']]
    df_temp_preflop_action_2_cbr_p5 = df_temp_preflop_action_2_cbr_p5.rename(columns = {'preflop_action_2_cbr':'preflop_action_2_cbr_p5'})
    df_temp_preflop_action_2_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'preflop_action_2_cbr']]
    df_temp_preflop_action_2_cbr_p6 = df_temp_preflop_action_2_cbr_p6.rename(columns = {'preflop_action_2_cbr':'preflop_action_2_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_preflop_action_2_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_preflop_action_2_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_preflop_action_2_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_preflop_action_2_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_preflop_action_2_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_preflop_action_2_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns preflop_action_3_cbr_pi
    df_temp_preflop_action_3_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'preflop_action_3_cbr']]
    df_temp_preflop_action_3_cbr_p1 = df_temp_preflop_action_3_cbr_p1.rename(columns = {'preflop_action_3_cbr':'preflop_action_3_cbr_p1'})
    df_temp_preflop_action_3_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'preflop_action_3_cbr']]
    df_temp_preflop_action_3_cbr_p2 = df_temp_preflop_action_3_cbr_p2.rename(columns = {'preflop_action_3_cbr':'preflop_action_3_cbr_p2'})
    df_temp_preflop_action_3_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'preflop_action_3_cbr']]
    df_temp_preflop_action_3_cbr_p3 = df_temp_preflop_action_3_cbr_p3.rename(columns = {'preflop_action_3_cbr':'preflop_action_3_cbr_p3'})
    df_temp_preflop_action_3_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'preflop_action_3_cbr']]
    df_temp_preflop_action_3_cbr_p4 = df_temp_preflop_action_3_cbr_p4.rename(columns = {'preflop_action_3_cbr':'preflop_action_3_cbr_p4'})
    df_temp_preflop_action_3_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'preflop_action_3_cbr']]
    df_temp_preflop_action_3_cbr_p5 = df_temp_preflop_action_3_cbr_p5.rename(columns = {'preflop_action_3_cbr':'preflop_action_3_cbr_p5'})
    df_temp_preflop_action_3_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'preflop_action_3_cbr']]
    df_temp_preflop_action_3_cbr_p6 = df_temp_preflop_action_3_cbr_p6.rename(columns = {'preflop_action_3_cbr':'preflop_action_3_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_preflop_action_3_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_preflop_action_3_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_preflop_action_3_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_preflop_action_3_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_preflop_action_3_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_preflop_action_3_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns flop_action_1_cbr_pi
    df_temp_flop_action_1_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'flop_action_1_cbr']]
    df_temp_flop_action_1_cbr_p1 = df_temp_flop_action_1_cbr_p1.rename(columns = {'flop_action_1_cbr':'flop_action_1_cbr_p1'})
    df_temp_flop_action_1_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'flop_action_1_cbr']]
    df_temp_flop_action_1_cbr_p2 = df_temp_flop_action_1_cbr_p2.rename(columns = {'flop_action_1_cbr':'flop_action_1_cbr_p2'})
    df_temp_flop_action_1_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'flop_action_1_cbr']]
    df_temp_flop_action_1_cbr_p3 = df_temp_flop_action_1_cbr_p3.rename(columns = {'flop_action_1_cbr':'flop_action_1_cbr_p3'})
    df_temp_flop_action_1_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'flop_action_1_cbr']]
    df_temp_flop_action_1_cbr_p4 = df_temp_flop_action_1_cbr_p4.rename(columns = {'flop_action_1_cbr':'flop_action_1_cbr_p4'})
    df_temp_flop_action_1_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'flop_action_1_cbr']]
    df_temp_flop_action_1_cbr_p5 = df_temp_flop_action_1_cbr_p5.rename(columns = {'flop_action_1_cbr':'flop_action_1_cbr_p5'})
    df_temp_flop_action_1_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'flop_action_1_cbr']]
    df_temp_flop_action_1_cbr_p6 = df_temp_flop_action_1_cbr_p6.rename(columns = {'flop_action_1_cbr':'flop_action_1_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_flop_action_1_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_flop_action_1_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_flop_action_1_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_flop_action_1_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_flop_action_1_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_flop_action_1_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns flop_action_2_cbr_pi
    df_temp_flop_action_2_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'flop_action_2_cbr']]
    df_temp_flop_action_2_cbr_p1 = df_temp_flop_action_2_cbr_p1.rename(columns = {'flop_action_2_cbr':'flop_action_2_cbr_p1'})
    df_temp_flop_action_2_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'flop_action_2_cbr']]
    df_temp_flop_action_2_cbr_p2 = df_temp_flop_action_2_cbr_p2.rename(columns = {'flop_action_2_cbr':'flop_action_2_cbr_p2'})
    df_temp_flop_action_2_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'flop_action_2_cbr']]
    df_temp_flop_action_2_cbr_p3 = df_temp_flop_action_2_cbr_p3.rename(columns = {'flop_action_2_cbr':'flop_action_2_cbr_p3'})
    df_temp_flop_action_2_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'flop_action_2_cbr']]
    df_temp_flop_action_2_cbr_p4 = df_temp_flop_action_2_cbr_p4.rename(columns = {'flop_action_2_cbr':'flop_action_2_cbr_p4'})
    df_temp_flop_action_2_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'flop_action_2_cbr']]
    df_temp_flop_action_2_cbr_p5 = df_temp_flop_action_2_cbr_p5.rename(columns = {'flop_action_2_cbr':'flop_action_2_cbr_p5'})
    df_temp_flop_action_2_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'flop_action_2_cbr']]
    df_temp_flop_action_2_cbr_p6 = df_temp_flop_action_2_cbr_p6.rename(columns = {'flop_action_2_cbr':'flop_action_2_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_flop_action_2_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_flop_action_2_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_flop_action_2_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_flop_action_2_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_flop_action_2_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_flop_action_2_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns flop_action_3_cbr_pi
    df_temp_flop_action_3_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'flop_action_3_cbr']]
    df_temp_flop_action_3_cbr_p1 = df_temp_flop_action_3_cbr_p1.rename(columns = {'flop_action_3_cbr':'flop_action_3_cbr_p1'})
    df_temp_flop_action_3_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'flop_action_3_cbr']]
    df_temp_flop_action_3_cbr_p2 = df_temp_flop_action_3_cbr_p2.rename(columns = {'flop_action_3_cbr':'flop_action_3_cbr_p2'})
    df_temp_flop_action_3_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'flop_action_3_cbr']]
    df_temp_flop_action_3_cbr_p3 = df_temp_flop_action_3_cbr_p3.rename(columns = {'flop_action_3_cbr':'flop_action_3_cbr_p3'})
    df_temp_flop_action_3_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'flop_action_3_cbr']]
    df_temp_flop_action_3_cbr_p4 = df_temp_flop_action_3_cbr_p4.rename(columns = {'flop_action_3_cbr':'flop_action_3_cbr_p4'})
    df_temp_flop_action_3_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'flop_action_3_cbr']]
    df_temp_flop_action_3_cbr_p5 = df_temp_flop_action_3_cbr_p5.rename(columns = {'flop_action_3_cbr':'flop_action_3_cbr_p5'})
    df_temp_flop_action_3_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'flop_action_3_cbr']]
    df_temp_flop_action_3_cbr_p6 = df_temp_flop_action_3_cbr_p6.rename(columns = {'flop_action_3_cbr':'flop_action_3_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_flop_action_3_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_flop_action_3_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_flop_action_3_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_flop_action_3_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_flop_action_3_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_flop_action_3_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns turn_action_1_cbr_pi
    df_temp_turn_action_1_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'turn_action_1_cbr']]
    df_temp_turn_action_1_cbr_p1 = df_temp_turn_action_1_cbr_p1.rename(columns = {'turn_action_1_cbr':'turn_action_1_cbr_p1'})
    df_temp_turn_action_1_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'turn_action_1_cbr']]
    df_temp_turn_action_1_cbr_p2 = df_temp_turn_action_1_cbr_p2.rename(columns = {'turn_action_1_cbr':'turn_action_1_cbr_p2'})
    df_temp_turn_action_1_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'turn_action_1_cbr']]
    df_temp_turn_action_1_cbr_p3 = df_temp_turn_action_1_cbr_p3.rename(columns = {'turn_action_1_cbr':'turn_action_1_cbr_p3'})
    df_temp_turn_action_1_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'turn_action_1_cbr']]
    df_temp_turn_action_1_cbr_p4 = df_temp_turn_action_1_cbr_p4.rename(columns = {'turn_action_1_cbr':'turn_action_1_cbr_p4'})
    df_temp_turn_action_1_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'turn_action_1_cbr']]
    df_temp_turn_action_1_cbr_p5 = df_temp_turn_action_1_cbr_p5.rename(columns = {'turn_action_1_cbr':'turn_action_1_cbr_p5'})
    df_temp_turn_action_1_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'turn_action_1_cbr']]
    df_temp_turn_action_1_cbr_p6 = df_temp_turn_action_1_cbr_p6.rename(columns = {'turn_action_1_cbr':'turn_action_1_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_turn_action_1_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_turn_action_1_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_turn_action_1_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_turn_action_1_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_turn_action_1_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_turn_action_1_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns turn_action_2_cbr_pi
    df_temp_turn_action_2_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'turn_action_2_cbr']]
    df_temp_turn_action_2_cbr_p1 = df_temp_turn_action_2_cbr_p1.rename(columns = {'turn_action_2_cbr':'turn_action_2_cbr_p1'})
    df_temp_turn_action_2_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'turn_action_2_cbr']]
    df_temp_turn_action_2_cbr_p2 = df_temp_turn_action_2_cbr_p2.rename(columns = {'turn_action_2_cbr':'turn_action_2_cbr_p2'})
    df_temp_turn_action_2_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'turn_action_2_cbr']]
    df_temp_turn_action_2_cbr_p3 = df_temp_turn_action_2_cbr_p3.rename(columns = {'turn_action_2_cbr':'turn_action_2_cbr_p3'})
    df_temp_turn_action_2_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'turn_action_2_cbr']]
    df_temp_turn_action_2_cbr_p4 = df_temp_turn_action_2_cbr_p4.rename(columns = {'turn_action_2_cbr':'turn_action_2_cbr_p4'})
    df_temp_turn_action_2_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'turn_action_2_cbr']]
    df_temp_turn_action_2_cbr_p5 = df_temp_turn_action_2_cbr_p5.rename(columns = {'turn_action_2_cbr':'turn_action_2_cbr_p5'})
    df_temp_turn_action_2_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'turn_action_2_cbr']]
    df_temp_turn_action_2_cbr_p6 = df_temp_turn_action_2_cbr_p6.rename(columns = {'turn_action_2_cbr':'turn_action_2_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_turn_action_2_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_turn_action_2_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_turn_action_2_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_turn_action_2_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_turn_action_2_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_turn_action_2_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns turn_action_3_cbr_pi
    df_temp_turn_action_3_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'turn_action_3_cbr']]
    df_temp_turn_action_3_cbr_p1 = df_temp_turn_action_3_cbr_p1.rename(columns = {'turn_action_3_cbr':'turn_action_3_cbr_p1'})
    df_temp_turn_action_3_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'turn_action_3_cbr']]
    df_temp_turn_action_3_cbr_p2 = df_temp_turn_action_3_cbr_p2.rename(columns = {'turn_action_3_cbr':'turn_action_3_cbr_p2'})
    df_temp_turn_action_3_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'turn_action_3_cbr']]
    df_temp_turn_action_3_cbr_p3 = df_temp_turn_action_3_cbr_p3.rename(columns = {'turn_action_3_cbr':'turn_action_3_cbr_p3'})
    df_temp_turn_action_3_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'turn_action_3_cbr']]
    df_temp_turn_action_3_cbr_p4 = df_temp_turn_action_3_cbr_p4.rename(columns = {'turn_action_3_cbr':'turn_action_3_cbr_p4'})
    df_temp_turn_action_3_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'turn_action_3_cbr']]
    df_temp_turn_action_3_cbr_p5 = df_temp_turn_action_3_cbr_p5.rename(columns = {'turn_action_3_cbr':'turn_action_3_cbr_p5'})
    df_temp_turn_action_3_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'turn_action_3_cbr']]
    df_temp_turn_action_3_cbr_p6 = df_temp_turn_action_3_cbr_p6.rename(columns = {'turn_action_3_cbr':'turn_action_3_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_turn_action_3_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_turn_action_3_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_turn_action_3_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_turn_action_3_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_turn_action_3_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_turn_action_3_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns river_action_1_cbr_pi
    df_temp_river_action_1_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'river_action_1_cbr']]
    df_temp_river_action_1_cbr_p1 = df_temp_river_action_1_cbr_p1.rename(columns = {'river_action_1_cbr':'river_action_1_cbr_p1'})
    df_temp_river_action_1_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'river_action_1_cbr']]
    df_temp_river_action_1_cbr_p2 = df_temp_river_action_1_cbr_p2.rename(columns = {'river_action_1_cbr':'river_action_1_cbr_p2'})
    df_temp_river_action_1_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'river_action_1_cbr']]
    df_temp_river_action_1_cbr_p3 = df_temp_river_action_1_cbr_p3.rename(columns = {'river_action_1_cbr':'river_action_1_cbr_p3'})
    df_temp_river_action_1_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'river_action_1_cbr']]
    df_temp_river_action_1_cbr_p4 = df_temp_river_action_1_cbr_p4.rename(columns = {'river_action_1_cbr':'river_action_1_cbr_p4'})
    df_temp_river_action_1_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'river_action_1_cbr']]
    df_temp_river_action_1_cbr_p5 = df_temp_river_action_1_cbr_p5.rename(columns = {'river_action_1_cbr':'river_action_1_cbr_p5'})
    df_temp_river_action_1_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'river_action_1_cbr']]
    df_temp_river_action_1_cbr_p6 = df_temp_river_action_1_cbr_p6.rename(columns = {'river_action_1_cbr':'river_action_1_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_river_action_1_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_river_action_1_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_river_action_1_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_river_action_1_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_river_action_1_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_river_action_1_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns river_action_2_cbr_pi
    df_temp_river_action_2_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'river_action_2_cbr']]
    df_temp_river_action_2_cbr_p1 = df_temp_river_action_2_cbr_p1.rename(columns = {'river_action_2_cbr':'river_action_2_cbr_p1'})
    df_temp_river_action_2_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'river_action_2_cbr']]
    df_temp_river_action_2_cbr_p2 = df_temp_river_action_2_cbr_p2.rename(columns = {'river_action_2_cbr':'river_action_2_cbr_p2'})
    df_temp_river_action_2_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'river_action_2_cbr']]
    df_temp_river_action_2_cbr_p3 = df_temp_river_action_2_cbr_p3.rename(columns = {'river_action_2_cbr':'river_action_2_cbr_p3'})
    df_temp_river_action_2_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'river_action_2_cbr']]
    df_temp_river_action_2_cbr_p4 = df_temp_river_action_2_cbr_p4.rename(columns = {'river_action_2_cbr':'river_action_2_cbr_p4'})
    df_temp_river_action_2_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'river_action_2_cbr']]
    df_temp_river_action_2_cbr_p5 = df_temp_river_action_2_cbr_p5.rename(columns = {'river_action_2_cbr':'river_action_2_cbr_p5'})
    df_temp_river_action_2_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'river_action_2_cbr']]
    df_temp_river_action_2_cbr_p6 = df_temp_river_action_2_cbr_p6.rename(columns = {'river_action_2_cbr':'river_action_2_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_river_action_2_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_river_action_2_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_river_action_2_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_river_action_2_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_river_action_2_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_river_action_2_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')
    
    # add columns river_action_3_cbr_pi
    df_temp_river_action_3_cbr_p1 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', ['game_id', 'river_action_3_cbr']]
    df_temp_river_action_3_cbr_p1 = df_temp_river_action_3_cbr_p1.rename(columns = {'river_action_3_cbr':'river_action_3_cbr_p1'})
    df_temp_river_action_3_cbr_p2 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', ['game_id', 'river_action_3_cbr']]
    df_temp_river_action_3_cbr_p2 = df_temp_river_action_3_cbr_p2.rename(columns = {'river_action_3_cbr':'river_action_3_cbr_p2'})
    df_temp_river_action_3_cbr_p3 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', ['game_id', 'river_action_3_cbr']]
    df_temp_river_action_3_cbr_p3 = df_temp_river_action_3_cbr_p3.rename(columns = {'river_action_3_cbr':'river_action_3_cbr_p3'})
    df_temp_river_action_3_cbr_p4 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', ['game_id', 'river_action_3_cbr']]
    df_temp_river_action_3_cbr_p4 = df_temp_river_action_3_cbr_p4.rename(columns = {'river_action_3_cbr':'river_action_3_cbr_p4'})
    df_temp_river_action_3_cbr_p5 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', ['game_id', 'river_action_3_cbr']]
    df_temp_river_action_3_cbr_p5 = df_temp_river_action_3_cbr_p5.rename(columns = {'river_action_3_cbr':'river_action_3_cbr_p5'})
    df_temp_river_action_3_cbr_p6 = df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', ['game_id', 'river_action_3_cbr']]
    df_temp_river_action_3_cbr_p6 = df_temp_river_action_3_cbr_p6.rename(columns = {'river_action_3_cbr':'river_action_3_cbr_p6'})
    
    df_temp_7 = pd.merge(left = df_games_players_enriched, right = df_temp_river_action_3_cbr_p1, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_8 = pd.merge(left = df_temp_7, right = df_temp_river_action_3_cbr_p2, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_9 = pd.merge(left = df_temp_8, right = df_temp_river_action_3_cbr_p3, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_10 = pd.merge(left = df_temp_9, right = df_temp_river_action_3_cbr_p4, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_temp_11 = pd.merge(left = df_temp_10, right = df_temp_river_action_3_cbr_p5, left_on = 'game_id', right_on = 'game_id', how = 'left')
    df_games_players_enriched = pd.merge(left = df_temp_11, right = df_temp_river_action_3_cbr_p6, left_on = 'game_id', right_on = 'game_id', how = 'left')

    # change type of some columns to integer
    df_games_players_enriched['small_blind'] = df_games_players_enriched['small_blind'].astype(int)
    df_games_players_enriched['big_blind'] = df_games_players_enriched['big_blind'].astype(int)
    
    df_games_players_enriched['preflop_action_1_cbr_p1'] = df_games_players_enriched['preflop_action_1_cbr_p1'].astype(int)
    df_games_players_enriched['preflop_action_1_cbr_p2'] = df_games_players_enriched['preflop_action_1_cbr_p2'].astype(int)
    df_games_players_enriched['preflop_action_1_cbr_p3'] = df_games_players_enriched['preflop_action_1_cbr_p3'].astype(int)
    df_games_players_enriched['preflop_action_1_cbr_p4'] = df_games_players_enriched['preflop_action_1_cbr_p4'].astype(int)
    df_games_players_enriched['preflop_action_1_cbr_p5'] = df_games_players_enriched['preflop_action_1_cbr_p5'].astype(int)
    df_games_players_enriched['preflop_action_1_cbr_p6'] = df_games_players_enriched['preflop_action_1_cbr_p6'].astype(int)
    
    df_games_players_enriched['preflop_action_2_cbr_p1'] = df_games_players_enriched['preflop_action_2_cbr_p1'].astype(int)
    df_games_players_enriched['preflop_action_2_cbr_p2'] = df_games_players_enriched['preflop_action_2_cbr_p2'].astype(int)
    df_games_players_enriched['preflop_action_2_cbr_p3'] = df_games_players_enriched['preflop_action_2_cbr_p3'].astype(int)
    df_games_players_enriched['preflop_action_2_cbr_p4'] = df_games_players_enriched['preflop_action_2_cbr_p4'].astype(int)
    df_games_players_enriched['preflop_action_2_cbr_p5'] = df_games_players_enriched['preflop_action_2_cbr_p5'].astype(int)
    df_games_players_enriched['preflop_action_2_cbr_p6'] = df_games_players_enriched['preflop_action_2_cbr_p6'].astype(int)
    
    df_games_players_enriched['preflop_action_3_cbr_p1'] = df_games_players_enriched['preflop_action_3_cbr_p1'].astype(int)
    df_games_players_enriched['preflop_action_3_cbr_p2'] = df_games_players_enriched['preflop_action_3_cbr_p2'].astype(int)
    df_games_players_enriched['preflop_action_3_cbr_p3'] = df_games_players_enriched['preflop_action_3_cbr_p3'].astype(int)
    df_games_players_enriched['preflop_action_3_cbr_p4'] = df_games_players_enriched['preflop_action_3_cbr_p4'].astype(int)
    df_games_players_enriched['preflop_action_3_cbr_p5'] = df_games_players_enriched['preflop_action_3_cbr_p5'].astype(int)
    df_games_players_enriched['preflop_action_3_cbr_p6'] = df_games_players_enriched['preflop_action_3_cbr_p6'].astype(int)
    
    df_games_players_enriched['flop_action_1_cbr_p1'] = df_games_players_enriched['flop_action_1_cbr_p1'].astype(int)
    df_games_players_enriched['flop_action_1_cbr_p2'] = df_games_players_enriched['flop_action_1_cbr_p2'].astype(int)
    df_games_players_enriched['flop_action_1_cbr_p3'] = df_games_players_enriched['flop_action_1_cbr_p3'].astype(int)
    df_games_players_enriched['flop_action_1_cbr_p4'] = df_games_players_enriched['flop_action_1_cbr_p4'].astype(int)
    df_games_players_enriched['flop_action_1_cbr_p5'] = df_games_players_enriched['flop_action_1_cbr_p5'].astype(int)
    df_games_players_enriched['flop_action_1_cbr_p6'] = df_games_players_enriched['flop_action_1_cbr_p6'].astype(int)
    
    df_games_players_enriched['flop_action_2_cbr_p1'] = df_games_players_enriched['flop_action_2_cbr_p1'].astype(int)
    df_games_players_enriched['flop_action_2_cbr_p2'] = df_games_players_enriched['flop_action_2_cbr_p2'].astype(int)
    df_games_players_enriched['flop_action_2_cbr_p3'] = df_games_players_enriched['flop_action_2_cbr_p3'].astype(int)
    df_games_players_enriched['flop_action_2_cbr_p4'] = df_games_players_enriched['flop_action_2_cbr_p4'].astype(int)
    df_games_players_enriched['flop_action_2_cbr_p5'] = df_games_players_enriched['flop_action_2_cbr_p5'].astype(int)
    df_games_players_enriched['flop_action_2_cbr_p6'] = df_games_players_enriched['flop_action_2_cbr_p6'].astype(int)
    
    df_games_players_enriched['flop_action_3_cbr_p1'] = df_games_players_enriched['flop_action_3_cbr_p1'].astype(int)
    df_games_players_enriched['flop_action_3_cbr_p2'] = df_games_players_enriched['flop_action_3_cbr_p2'].astype(int)
    df_games_players_enriched['flop_action_3_cbr_p3'] = df_games_players_enriched['flop_action_3_cbr_p3'].astype(int)
    df_games_players_enriched['flop_action_3_cbr_p4'] = df_games_players_enriched['flop_action_3_cbr_p4'].astype(int)
    df_games_players_enriched['flop_action_3_cbr_p5'] = df_games_players_enriched['flop_action_3_cbr_p5'].astype(int)
    df_games_players_enriched['flop_action_3_cbr_p6'] = df_games_players_enriched['flop_action_3_cbr_p6'].astype(int)
    
    df_games_players_enriched['turn_action_1_cbr_p1'] = df_games_players_enriched['turn_action_1_cbr_p1'].astype(int)
    df_games_players_enriched['turn_action_1_cbr_p2'] = df_games_players_enriched['turn_action_1_cbr_p2'].astype(int)
    df_games_players_enriched['turn_action_1_cbr_p3'] = df_games_players_enriched['turn_action_1_cbr_p3'].astype(int)
    df_games_players_enriched['turn_action_1_cbr_p4'] = df_games_players_enriched['turn_action_1_cbr_p4'].astype(int)
    df_games_players_enriched['turn_action_1_cbr_p5'] = df_games_players_enriched['turn_action_1_cbr_p5'].astype(int)
    df_games_players_enriched['turn_action_1_cbr_p6'] = df_games_players_enriched['turn_action_1_cbr_p6'].astype(int)
    
    df_games_players_enriched['turn_action_2_cbr_p1'] = df_games_players_enriched['turn_action_2_cbr_p1'].astype(int)
    df_games_players_enriched['turn_action_2_cbr_p2'] = df_games_players_enriched['turn_action_2_cbr_p2'].astype(int)
    df_games_players_enriched['turn_action_2_cbr_p3'] = df_games_players_enriched['turn_action_2_cbr_p3'].astype(int)
    df_games_players_enriched['turn_action_2_cbr_p4'] = df_games_players_enriched['turn_action_2_cbr_p4'].astype(int)
    df_games_players_enriched['turn_action_2_cbr_p5'] = df_games_players_enriched['turn_action_2_cbr_p5'].astype(int)
    df_games_players_enriched['turn_action_2_cbr_p6'] = df_games_players_enriched['turn_action_2_cbr_p6'].astype(int)
    
    df_games_players_enriched['turn_action_3_cbr_p1'] = df_games_players_enriched['turn_action_3_cbr_p1'].astype(int)
    df_games_players_enriched['turn_action_3_cbr_p2'] = df_games_players_enriched['turn_action_3_cbr_p2'].astype(int)
    df_games_players_enriched['turn_action_3_cbr_p3'] = df_games_players_enriched['turn_action_3_cbr_p3'].astype(int)
    df_games_players_enriched['turn_action_3_cbr_p4'] = df_games_players_enriched['turn_action_3_cbr_p4'].astype(int)
    df_games_players_enriched['turn_action_3_cbr_p5'] = df_games_players_enriched['turn_action_3_cbr_p5'].astype(int)
    df_games_players_enriched['turn_action_3_cbr_p6'] = df_games_players_enriched['turn_action_3_cbr_p6'].astype(int)
    
    df_games_players_enriched['river_action_1_cbr_p1'] = df_games_players_enriched['river_action_1_cbr_p1'].astype(int)
    df_games_players_enriched['river_action_1_cbr_p2'] = df_games_players_enriched['river_action_1_cbr_p2'].astype(int)
    df_games_players_enriched['river_action_1_cbr_p3'] = df_games_players_enriched['river_action_1_cbr_p3'].astype(int)
    df_games_players_enriched['river_action_1_cbr_p4'] = df_games_players_enriched['river_action_1_cbr_p4'].astype(int)
    df_games_players_enriched['river_action_1_cbr_p5'] = df_games_players_enriched['river_action_1_cbr_p5'].astype(int)
    df_games_players_enriched['river_action_1_cbr_p6'] = df_games_players_enriched['river_action_1_cbr_p6'].astype(int)
    
    df_games_players_enriched['river_action_2_cbr_p1'] = df_games_players_enriched['river_action_2_cbr_p1'].astype(int)
    df_games_players_enriched['river_action_2_cbr_p2'] = df_games_players_enriched['river_action_2_cbr_p2'].astype(int)
    df_games_players_enriched['river_action_2_cbr_p3'] = df_games_players_enriched['river_action_2_cbr_p3'].astype(int)
    df_games_players_enriched['river_action_2_cbr_p4'] = df_games_players_enriched['river_action_2_cbr_p4'].astype(int)
    df_games_players_enriched['river_action_2_cbr_p5'] = df_games_players_enriched['river_action_2_cbr_p5'].astype(int)
    df_games_players_enriched['river_action_2_cbr_p6'] = df_games_players_enriched['river_action_2_cbr_p6'].astype(int)
    
    df_games_players_enriched['river_action_3_cbr_p1'] = df_games_players_enriched['river_action_3_cbr_p1'].astype(int)
    df_games_players_enriched['river_action_3_cbr_p2'] = df_games_players_enriched['river_action_3_cbr_p2'].astype(int)
    df_games_players_enriched['river_action_3_cbr_p3'] = df_games_players_enriched['river_action_3_cbr_p3'].astype(int)
    df_games_players_enriched['river_action_3_cbr_p4'] = df_games_players_enriched['river_action_3_cbr_p4'].astype(int)
    df_games_players_enriched['river_action_3_cbr_p5'] = df_games_players_enriched['river_action_3_cbr_p5'].astype(int)
    df_games_players_enriched['river_action_3_cbr_p6'] = df_games_players_enriched['river_action_3_cbr_p6'].astype(int)

    # add columns cbr_already_in
    df_games_players_enriched['preflop_action_1_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_1_cbr_already_in'] = df_games_players_enriched['small_blind']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_1_cbr_already_in'] = df_games_players_enriched['big_blind']
    
    df_games_players_enriched['preflop_action_2_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_2_cbr_already_in'] = df_games_players_enriched[['preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_2_cbr_already_in'] = df_games_players_enriched[['preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_2_cbr_already_in'] = df_games_players_enriched[['preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_2_cbr_already_in'] = df_games_players_enriched[['preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_2_cbr_already_in'] = df_games_players_enriched[['preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_2_cbr_already_in'] = df_games_players_enriched[['preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    
    df_games_players_enriched['preflop_action_3_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_3_cbr_already_in'] = df_games_players_enriched[['preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p3', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p6', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1) 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_3_cbr_already_in'] = df_games_players_enriched[['preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p3', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p6', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1) 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_3_cbr_already_in'] = df_games_players_enriched[['preflop_action_2_cbr_p3', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p6', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1) 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_3_cbr_already_in'] = df_games_players_enriched[['preflop_action_2_cbr_p3', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p5', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1) 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_3_cbr_already_in'] = df_games_players_enriched[['preflop_action_2_cbr_p3', 'preflop_action_2_cbr_p4', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_3_cbr_already_in'] = df_games_players_enriched[['preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6', 'big_blind']].max(axis=1) 
    
    df_games_players_enriched['flop_action_1_cbr_already_in'] = df_games_players_enriched[['preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p3', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p6',
                                         'preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p3', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p6',
                                         'preflop_action_3_cbr_p1', 'preflop_action_3_cbr_p2', 'preflop_action_3_cbr_p3', 'preflop_action_3_cbr_p4', 'preflop_action_3_cbr_p5', 'preflop_action_3_cbr_p6', 'big_blind']].max(axis=1)
    
    df_games_players_enriched['flop_action_2_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_2_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_2_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_2_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_2_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_2_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_2_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in']
    
    df_games_players_enriched['flop_action_3_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_3_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6', 'flop_action_2_cbr_p1', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p6']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_3_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6', 'flop_action_2_cbr_p1', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p5']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_3_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6', 'flop_action_2_cbr_p1', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p4']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_3_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6', 'flop_action_2_cbr_p1', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p3']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_3_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6', 'flop_action_2_cbr_p1', 'flop_action_2_cbr_p2']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_3_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6', 'flop_action_2_cbr_p1']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in'] 
    
    df_games_players_enriched['turn_action_1_cbr_already_in'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p6',
                                         'flop_action_2_cbr_p1', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p6',
                                         'flop_action_3_cbr_p1', 'flop_action_3_cbr_p2', 'flop_action_3_cbr_p3', 'flop_action_3_cbr_p4', 'flop_action_3_cbr_p5', 'flop_action_3_cbr_p6']].max(axis=1) + df_games_players_enriched['flop_action_1_cbr_already_in']
    
    df_games_players_enriched['turn_action_2_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_2_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_2_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_2_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_2_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_2_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_2_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in']
    
    df_games_players_enriched['turn_action_3_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_3_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6', 'turn_action_2_cbr_p1', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p6']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_3_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6', 'turn_action_2_cbr_p1', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p5']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_3_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6', 'turn_action_2_cbr_p1', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p4']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_3_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6', 'turn_action_2_cbr_p1', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p3']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_3_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6', 'turn_action_2_cbr_p1', 'turn_action_2_cbr_p2']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_3_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6', 'turn_action_2_cbr_p1']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in'] 
    
    df_games_players_enriched['river_action_1_cbr_already_in'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p6',
                                         'turn_action_2_cbr_p1', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p6',
                                         'turn_action_3_cbr_p1', 'turn_action_3_cbr_p2', 'turn_action_3_cbr_p3', 'turn_action_3_cbr_p4', 'turn_action_3_cbr_p5', 'turn_action_3_cbr_p6']].max(axis=1) + df_games_players_enriched['turn_action_1_cbr_already_in']
    
    df_games_players_enriched['river_action_2_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_2_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5', 'river_action_1_cbr_p6']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_2_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_2_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_2_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_2_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_2_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in']
    
    df_games_players_enriched['river_action_3_cbr_already_in'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_3_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5', 'river_action_1_cbr_p6', 'river_action_2_cbr_p1', 'river_action_2_cbr_p2', 'river_action_2_cbr_p3', 'river_action_2_cbr_p4', 'river_action_2_cbr_p5', 'river_action_2_cbr_p6']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_3_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5', 'river_action_1_cbr_p6', 'river_action_2_cbr_p1', 'river_action_2_cbr_p2', 'river_action_2_cbr_p3', 'river_action_2_cbr_p4', 'river_action_2_cbr_p5']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_3_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5', 'river_action_1_cbr_p6', 'river_action_2_cbr_p1', 'river_action_2_cbr_p2', 'river_action_2_cbr_p3', 'river_action_2_cbr_p4']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_3_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5', 'river_action_1_cbr_p6', 'river_action_2_cbr_p1', 'river_action_2_cbr_p2', 'river_action_2_cbr_p3']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_3_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5', 'river_action_1_cbr_p6', 'river_action_2_cbr_p1', 'river_action_2_cbr_p2']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in'] 
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_3_cbr_already_in'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_p2', 'river_action_1_cbr_p3', 'river_action_1_cbr_p4', 'river_action_1_cbr_p5', 'river_action_1_cbr_p6', 'river_action_2_cbr_p1']].max(axis=1) + df_games_players_enriched['river_action_1_cbr_already_in'] 
    
    # add columns cbr_to_continue
    df_games_players_enriched['preflop_action_1_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_1_cbr_to_continue'] = df_games_players_enriched['big_blind']
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_1_cbr_to_continue'] = df_games_players_enriched[['preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_1_cbr_to_continue'] = df_games_players_enriched[['preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_1_cbr_to_continue'] = df_games_players_enriched[['preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_1_cbr_to_continue'] = df_games_players_enriched[['preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_1_cbr_to_continue'] = df_games_players_enriched[['preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched['preflop_action_1_cbr_to_continue'] = df_games_players_enriched['preflop_action_1_cbr_to_continue'] - df_games_players_enriched['preflop_action_1_cbr_already_in']
    
    df_games_players_enriched['preflop_action_2_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_2_cbr_to_continue'] = df_games_players_enriched[['preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_2_cbr_to_continue'] = df_games_players_enriched[['preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_2_cbr_to_continue'] = df_games_players_enriched[['preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_2_cbr_to_continue'] = df_games_players_enriched[['preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_2_cbr_to_continue'] = df_games_players_enriched[['preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_2_cbr_to_continue'] = df_games_players_enriched[['preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched['preflop_action_2_cbr_to_continue'] = df_games_players_enriched['preflop_action_2_cbr_to_continue'] - df_games_players_enriched['preflop_action_2_cbr_already_in']
    
    df_games_players_enriched['preflop_action_3_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'preflop_action_3_cbr_to_continue'] = df_games_players_enriched[['preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'preflop_action_3_cbr_to_continue'] = df_games_players_enriched[['preflop_action_3_cbr_p3', 'preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'preflop_action_3_cbr_to_continue'] = df_games_players_enriched[['preflop_action_3_cbr_p4', 'preflop_action_3_cbr_p3', 'preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'preflop_action_3_cbr_to_continue'] = df_games_players_enriched[['preflop_action_3_cbr_p5', 'preflop_action_3_cbr_p4', 'preflop_action_3_cbr_p3', 'preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'preflop_action_3_cbr_to_continue'] = df_games_players_enriched[['preflop_action_3_cbr_p6', 'preflop_action_3_cbr_p5', 'preflop_action_3_cbr_p4', 'preflop_action_3_cbr_p3', 'preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'preflop_action_3_cbr_to_continue'] = df_games_players_enriched[['preflop_action_3_cbr_p1', 'preflop_action_3_cbr_p6', 'preflop_action_3_cbr_p5', 'preflop_action_3_cbr_p4', 'preflop_action_3_cbr_p3', 'preflop_action_2_cbr_p2', 'preflop_action_2_cbr_p1', 'preflop_action_2_cbr_p6', 'preflop_action_2_cbr_p5', 'preflop_action_2_cbr_p4', 'preflop_action_2_cbr_p3', 'preflop_action_1_cbr_p2', 'preflop_action_1_cbr_p1', 'preflop_action_1_cbr_p6', 'preflop_action_1_cbr_p5', 'preflop_action_1_cbr_p4', 'preflop_action_1_cbr_p3', 'big_blind']].max(axis=1)
    df_games_players_enriched['preflop_action_3_cbr_to_continue'] = df_games_players_enriched['preflop_action_3_cbr_to_continue'] - df_games_players_enriched['preflop_action_3_cbr_already_in']
    
    df_games_players_enriched['flop_action_1_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_1_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_1_cbr_to_continue'] = df_games_players_enriched[['flop_action_1_cbr_p1', 'flop_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_1_cbr_to_continue'] = df_games_players_enriched[['flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_1_cbr_to_continue'] = df_games_players_enriched[['flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_1_cbr_to_continue'] = df_games_players_enriched[['flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_1_cbr_to_continue'] = df_games_players_enriched[['flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_1_cbr_to_continue']].max(axis=1)
    
    df_games_players_enriched['flop_action_2_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_2_cbr_to_continue'] = df_games_players_enriched[['flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_2_cbr_to_continue'] = df_games_players_enriched[['flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_2_cbr_to_continue'] = df_games_players_enriched[['flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_2_cbr_to_continue'] = df_games_players_enriched[['flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_2_cbr_to_continue'] = df_games_players_enriched[['flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_2_cbr_to_continue'] = df_games_players_enriched[['flop_action_2_cbr_p5', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched['flop_action_2_cbr_to_continue'] = df_games_players_enriched['flop_action_2_cbr_to_continue'] - df_games_players_enriched['flop_action_2_cbr_already_in'] + df_games_players_enriched['flop_action_1_cbr_already_in']
    
    df_games_players_enriched['flop_action_3_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'flop_action_3_cbr_to_continue'] = df_games_players_enriched[['flop_action_2_cbr_p6', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'flop_action_3_cbr_to_continue'] = df_games_players_enriched[['flop_action_3_cbr_p1', 'flop_action_2_cbr_p6', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'flop_action_3_cbr_to_continue'] = df_games_players_enriched[['flop_action_3_cbr_p2', 'flop_action_3_cbr_p1', 'flop_action_2_cbr_p6', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'flop_action_3_cbr_to_continue'] = df_games_players_enriched[['flop_action_3_cbr_p3', 'flop_action_3_cbr_p2', 'flop_action_3_cbr_p1', 'flop_action_2_cbr_p6', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'flop_action_3_cbr_to_continue'] = df_games_players_enriched[['flop_action_3_cbr_p4', 'flop_action_3_cbr_p3', 'flop_action_3_cbr_p2', 'flop_action_3_cbr_p1', 'flop_action_2_cbr_p6', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'flop_action_3_cbr_to_continue'] = df_games_players_enriched[['flop_action_3_cbr_p5', 'flop_action_3_cbr_p4', 'flop_action_3_cbr_p3', 'flop_action_3_cbr_p2', 'flop_action_3_cbr_p1', 'flop_action_2_cbr_p6', 'flop_action_2_cbr_p5', 'flop_action_2_cbr_p4', 'flop_action_2_cbr_p3', 'flop_action_2_cbr_p2', 'flop_action_2_cbr_p1', 'flop_action_1_cbr_p6', 'flop_action_1_cbr_p5', 'flop_action_1_cbr_p4', 'flop_action_1_cbr_p3', 'flop_action_1_cbr_p2', 'flop_action_1_cbr_p1', 'flop_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched['flop_action_3_cbr_to_continue'] = df_games_players_enriched['flop_action_3_cbr_to_continue'] - df_games_players_enriched['flop_action_3_cbr_already_in'] + df_games_players_enriched['flop_action_1_cbr_already_in']
    
    df_games_players_enriched['turn_action_1_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_1_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_1_cbr_to_continue'] = df_games_players_enriched[['turn_action_1_cbr_p1', 'turn_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_1_cbr_to_continue'] = df_games_players_enriched[['turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_1_cbr_to_continue'] = df_games_players_enriched[['turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_1_cbr_to_continue'] = df_games_players_enriched[['turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_1_cbr_to_continue'] = df_games_players_enriched[['turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_1_cbr_to_continue']].max(axis=1)
    
    df_games_players_enriched['turn_action_2_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_2_cbr_to_continue'] = df_games_players_enriched[['turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_2_cbr_to_continue'] = df_games_players_enriched[['turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_2_cbr_to_continue'] = df_games_players_enriched[['turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_2_cbr_to_continue'] = df_games_players_enriched[['turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_2_cbr_to_continue'] = df_games_players_enriched[['turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_2_cbr_to_continue'] = df_games_players_enriched[['turn_action_2_cbr_p5', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched['turn_action_2_cbr_to_continue'] = df_games_players_enriched['turn_action_2_cbr_to_continue'] - df_games_players_enriched['turn_action_2_cbr_already_in'] + df_games_players_enriched['turn_action_1_cbr_already_in']
    
    df_games_players_enriched['turn_action_3_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'turn_action_3_cbr_to_continue'] = df_games_players_enriched[['turn_action_2_cbr_p6', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'turn_action_3_cbr_to_continue'] = df_games_players_enriched[['turn_action_3_cbr_p1', 'turn_action_2_cbr_p6', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'turn_action_3_cbr_to_continue'] = df_games_players_enriched[['turn_action_3_cbr_p2', 'turn_action_3_cbr_p1', 'turn_action_2_cbr_p6', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'turn_action_3_cbr_to_continue'] = df_games_players_enriched[['turn_action_3_cbr_p3', 'turn_action_3_cbr_p2', 'turn_action_3_cbr_p1', 'turn_action_2_cbr_p6', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'turn_action_3_cbr_to_continue'] = df_games_players_enriched[['turn_action_3_cbr_p4', 'turn_action_3_cbr_p3', 'turn_action_3_cbr_p2', 'turn_action_3_cbr_p1', 'turn_action_2_cbr_p6', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'turn_action_3_cbr_to_continue'] = df_games_players_enriched[['turn_action_3_cbr_p5', 'turn_action_3_cbr_p4', 'turn_action_3_cbr_p3', 'turn_action_3_cbr_p2', 'turn_action_3_cbr_p1', 'turn_action_2_cbr_p6', 'turn_action_2_cbr_p5', 'turn_action_2_cbr_p4', 'turn_action_2_cbr_p3', 'turn_action_2_cbr_p2', 'turn_action_2_cbr_p1', 'turn_action_1_cbr_p6', 'turn_action_1_cbr_p5', 'turn_action_1_cbr_p4', 'turn_action_1_cbr_p3', 'turn_action_1_cbr_p2', 'turn_action_1_cbr_p1', 'turn_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched['turn_action_3_cbr_to_continue'] = df_games_players_enriched['turn_action_3_cbr_to_continue'] - df_games_players_enriched['turn_action_3_cbr_already_in'] + df_games_players_enriched['turn_action_1_cbr_already_in']
    
    df_games_players_enriched['river_action_1_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_1_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_1_cbr_to_continue'] = df_games_players_enriched[['river_action_1_cbr_p1', 'river_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_1_cbr_to_continue'] = df_games_players_enriched[['river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_1_cbr_to_continue'] = df_games_players_enriched[['river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_1_cbr_to_continue'] = df_games_players_enriched[['river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_1_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_1_cbr_to_continue'] = df_games_players_enriched[['river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_1_cbr_to_continue']].max(axis=1)
    
    df_games_players_enriched['river_action_2_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_2_cbr_to_continue'] = df_games_players_enriched[['river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_2_cbr_to_continue'] = df_games_players_enriched[['river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_2_cbr_to_continue'] = df_games_players_enriched[['river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_2_cbr_to_continue'] = df_games_players_enriched[['river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_2_cbr_to_continue'] = df_games_players_enriched[['river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_2_cbr_to_continue'] = df_games_players_enriched[['river_action_2_cbr_p5', 'river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_2_cbr_to_continue']].max(axis=1)
    df_games_players_enriched['river_action_2_cbr_to_continue'] = df_games_players_enriched['river_action_2_cbr_to_continue'] - df_games_players_enriched['river_action_2_cbr_already_in'] + df_games_players_enriched['river_action_1_cbr_already_in']
    
    df_games_players_enriched['river_action_3_cbr_to_continue'] = 0
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p1', 'river_action_3_cbr_to_continue'] = df_games_players_enriched[['river_action_2_cbr_p6', 'river_action_2_cbr_p5', 'river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p2', 'river_action_3_cbr_to_continue'] = df_games_players_enriched[['river_action_3_cbr_p1', 'river_action_2_cbr_p6', 'river_action_2_cbr_p5', 'river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p3', 'river_action_3_cbr_to_continue'] = df_games_players_enriched[['river_action_3_cbr_p2', 'river_action_3_cbr_p1', 'river_action_2_cbr_p6', 'river_action_2_cbr_p5', 'river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p4', 'river_action_3_cbr_to_continue'] = df_games_players_enriched[['river_action_3_cbr_p3', 'river_action_3_cbr_p2', 'river_action_3_cbr_p1', 'river_action_2_cbr_p6', 'river_action_2_cbr_p5', 'river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p5', 'river_action_3_cbr_to_continue'] = df_games_players_enriched[['river_action_3_cbr_p4', 'river_action_3_cbr_p3', 'river_action_3_cbr_p2', 'river_action_3_cbr_p1', 'river_action_2_cbr_p6', 'river_action_2_cbr_p5', 'river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched.loc[df_games_players_enriched['player_id'] == 'p6', 'river_action_3_cbr_to_continue'] = df_games_players_enriched[['river_action_3_cbr_p5', 'river_action_3_cbr_p4', 'river_action_3_cbr_p3', 'river_action_3_cbr_p2', 'river_action_3_cbr_p1', 'river_action_2_cbr_p6', 'river_action_2_cbr_p5', 'river_action_2_cbr_p4', 'river_action_2_cbr_p3', 'river_action_2_cbr_p2', 'river_action_2_cbr_p1', 'river_action_1_cbr_p6', 'river_action_1_cbr_p5', 'river_action_1_cbr_p4', 'river_action_1_cbr_p3', 'river_action_1_cbr_p2', 'river_action_1_cbr_p1', 'river_action_3_cbr_to_continue']].max(axis=1)
    df_games_players_enriched['river_action_3_cbr_to_continue'] = df_games_players_enriched['river_action_3_cbr_to_continue'] - df_games_players_enriched['river_action_3_cbr_already_in'] + df_games_players_enriched['river_action_1_cbr_already_in']

    # update datatype of some columns to integer
    df_games_players_enriched['zero'] = 0
    df_games_players_enriched['preflop_action_1_cbr'] = df_games_players_enriched['preflop_action_1_cbr'].astype(int)
    df_games_players_enriched['preflop_action_2_cbr'] = df_games_players_enriched['preflop_action_2_cbr'].astype(int)
    df_games_players_enriched['preflop_action_3_cbr'] = df_games_players_enriched['preflop_action_3_cbr'].astype(int)
    df_games_players_enriched['flop_action_1_cbr'] = df_games_players_enriched['flop_action_1_cbr'].astype(int)
    df_games_players_enriched['flop_action_2_cbr'] = df_games_players_enriched['flop_action_2_cbr'].astype(int)
    df_games_players_enriched['flop_action_3_cbr'] = df_games_players_enriched['flop_action_3_cbr'].astype(int)
    df_games_players_enriched['turn_action_1_cbr'] = df_games_players_enriched['turn_action_1_cbr'].astype(int)
    df_games_players_enriched['turn_action_2_cbr'] = df_games_players_enriched['turn_action_2_cbr'].astype(int)
    df_games_players_enriched['turn_action_3_cbr'] = df_games_players_enriched['turn_action_3_cbr'].astype(int)
    df_games_players_enriched['river_action_1_cbr'] = df_games_players_enriched['river_action_1_cbr'].astype(int)
    df_games_players_enriched['river_action_2_cbr'] = df_games_players_enriched['river_action_2_cbr'].astype(int)
    df_games_players_enriched['river_action_3_cbr'] = df_games_players_enriched['river_action_3_cbr'].astype(int)
    
    # add columns cbr_delta
    df_games_players_enriched['preflop_action_1_cbr_delta'] = df_games_players_enriched['preflop_action_1_cbr'] - df_games_players_enriched['preflop_action_1_cbr_already_in']
    df_games_players_enriched['preflop_action_1_cbr_delta'] = df_games_players_enriched[['preflop_action_1_cbr_delta', 'zero']].max(axis=1)
    df_games_players_enriched['preflop_action_2_cbr_delta'] = df_games_players_enriched['preflop_action_2_cbr'] - df_games_players_enriched['preflop_action_2_cbr_already_in']
    df_games_players_enriched['preflop_action_2_cbr_delta'] = df_games_players_enriched[['preflop_action_2_cbr_delta', 'zero']].max(axis=1)
    df_games_players_enriched['preflop_action_3_cbr_delta'] = df_games_players_enriched['preflop_action_3_cbr'] - df_games_players_enriched['preflop_action_3_cbr_already_in']
    df_games_players_enriched['preflop_action_3_cbr_delta'] = df_games_players_enriched[['preflop_action_3_cbr_delta', 'zero']].max(axis=1)
    
    df_games_players_enriched['flop_action_1_cbr_delta'] = df_games_players_enriched['flop_action_1_cbr']
    df_games_players_enriched['flop_action_2_cbr_delta'] = df_games_players_enriched['flop_action_2_cbr'] - df_games_players_enriched['flop_action_2_cbr_already_in'] + df_games_players_enriched['flop_action_1_cbr_already_in']
    df_games_players_enriched['flop_action_2_cbr_delta'] = df_games_players_enriched[['flop_action_2_cbr_delta', 'zero']].max(axis=1)
    df_games_players_enriched['flop_action_3_cbr_delta'] = df_games_players_enriched['flop_action_3_cbr'] - df_games_players_enriched['flop_action_3_cbr_already_in'] + df_games_players_enriched['flop_action_1_cbr_already_in']
    df_games_players_enriched['flop_action_3_cbr_delta'] = df_games_players_enriched[['flop_action_3_cbr_delta', 'zero']].max(axis=1)
    
    df_games_players_enriched['turn_action_1_cbr_delta'] = df_games_players_enriched['turn_action_1_cbr']
    df_games_players_enriched['turn_action_2_cbr_delta'] = df_games_players_enriched['turn_action_2_cbr'] - df_games_players_enriched['turn_action_2_cbr_already_in'] + df_games_players_enriched['turn_action_1_cbr_already_in']
    df_games_players_enriched['turn_action_2_cbr_delta'] = df_games_players_enriched[['turn_action_2_cbr_delta', 'zero']].max(axis=1)
    df_games_players_enriched['turn_action_3_cbr_delta'] = df_games_players_enriched['turn_action_3_cbr'] - df_games_players_enriched['turn_action_3_cbr_already_in'] + df_games_players_enriched['turn_action_1_cbr_already_in']
    df_games_players_enriched['turn_action_3_cbr_delta'] = df_games_players_enriched[['turn_action_3_cbr_delta', 'zero']].max(axis=1)
    
    df_games_players_enriched['river_action_1_cbr_delta'] = df_games_players_enriched['river_action_1_cbr']
    df_games_players_enriched['river_action_2_cbr_delta'] = df_games_players_enriched['river_action_2_cbr'] - df_games_players_enriched['river_action_2_cbr_already_in'] + df_games_players_enriched['river_action_1_cbr_already_in']
    df_games_players_enriched['river_action_2_cbr_delta'] = df_games_players_enriched[['river_action_2_cbr_delta', 'zero']].max(axis=1)
    df_games_players_enriched['river_action_3_cbr_delta'] = df_games_players_enriched['river_action_3_cbr'] - df_games_players_enriched['river_action_3_cbr_already_in'] + df_games_players_enriched['river_action_1_cbr_already_in']
    df_games_players_enriched['river_action_3_cbr_delta'] = df_games_players_enriched[['river_action_3_cbr_delta', 'zero']].max(axis=1)

    return df_games_players_enriched

## Function formatTurnsDataframe
This function reads a dataframe of Poker Games aggregated by Game ID, Player ID and Turn ID and update its format by renaming some columns and changing its order

In [8]:
def formatTurnsDataframe(df_games_turns):
    """
    This function reads a dataframe of Poker Games aggregated by Game ID, Player ID and Turn ID
    and update its format by renaming some columns and changing its order
    
    Arguments
    ---------
    df_games_turns: pd.DataFrame
           Array aggregated by Game ID, Player ID and Turn ID.

    Returns
    --------
    df_games_turns_formatted: pd.DataFrame
         Array aggregated by Game ID, Player ID and Turn ID. It was formatted by renaming some columns and changing the order of some columns
    """
    # remove columns with static data or not very meaningful
    # small_blind is always 50
    # big_blind is always 100
    # min_bet is always 100
    # starting_stack is always 10000
    # sm and turn_id are not very meaningful (the turn_id will be recalculated below)
    df_games_turns_formatted = df_games_turns.drop(columns=['small_blind', 'big_blind', 'min_bet', 'starting_stack', 'sm', 'turn_id'])
    # rename some columns to have more meaningful name
    df_games_turns_formatted = df_games_turns_formatted.rename(columns = {'action_raise_amount':'amount_raised', 'amount_already_in':'amount_committed', 'amount_to_continue':'amount_required', 'finishing_stack':'amount_finish', 'd_dh':'cards', 'flop_d_db':'flop', 'turn_d_db':'turn', 'river_d_db':'river'})
    # change type of some columns to integer
    df_games_turns_formatted['nb_players_left'] = df_games_turns_formatted['nb_players_left'].astype(int)
    df_games_turns_formatted['amount_finish'] = df_games_turns_formatted['amount_finish'].astype(float)
    df_games_turns_formatted['amount_raised'] = df_games_turns_formatted['amount_raised'].astype(int)
    df_games_turns_formatted['amount_committed'] = df_games_turns_formatted['amount_committed'].astype(int)
    df_games_turns_formatted['amount_required'] = df_games_turns_formatted['amount_required'].astype(int)
    df_games_turns_formatted['nb_previous_raise'] = df_games_turns_formatted['nb_previous_raise'].astype(int)
    # replace repetitive values in column game_id
    df_games_turns_formatted['game_id'] = df_games_turns_formatted['game_id'].str.replace('__', '_', regex=False)
    # replace values in column action
    df_games_turns_formatted['action'] = df_games_turns_formatted['action'].str.replace('cbr', 'R', regex=False)
    df_games_turns_formatted['action'] = df_games_turns_formatted['action'].str.replace('f', 'F', regex=False)
    df_games_turns_formatted['action'] = df_games_turns_formatted['action'].str.replace('cc', 'C', regex=False)

    # update the sorting using an updated column turn_id
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['sub_stage'] + df_games_turns_formatted['player_id']
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_1p3', '1', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_1p4', '2', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_1p5', '3', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_1p6', '4', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_1p1', '5', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_1p2', '6', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_2p3', '7', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_2p4', '8', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_2p5', '9', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_2p6', '10', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_2p1', '11', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_2p2', '12', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_3p3', '13', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_3p4', '14', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_3p5', '15', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_3p6', '16', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_3p1', '17', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('preflop_3p2', '18', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_1p1', '19', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_1p2', '20', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_1p3', '21', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_1p4', '22', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_1p5', '23', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_1p6', '24', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_2p1', '25', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_2p2', '26', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_2p3', '27', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_2p4', '28', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_2p5', '29', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_2p6', '30', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_3p1', '31', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_3p2', '32', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_3p3', '33', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_3p4', '34', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_3p5', '35', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('flop_3p6', '36', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_1p1', '37', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_1p2', '38', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_1p3', '39', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_1p4', '40', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_1p5', '41', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_1p6', '42', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_2p1', '43', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_2p2', '44', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_2p3', '45', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_2p4', '46', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_2p5', '47', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_2p6', '48', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_3p1', '49', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_3p2', '50', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_3p3', '51', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_3p4', '52', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_3p5', '53', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('turn_3p6', '54', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_1p1', '55', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_1p2', '56', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_1p3', '57', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_1p4', '58', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_1p5', '59', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_1p6', '60', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_2p1', '61', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_2p2', '62', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_2p3', '63', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_2p4', '64', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_2p5', '65', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_2p6', '66', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_3p1', '67', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_3p2', '68', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_3p3', '69', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_3p4', '70', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_3p5', '71', regex=False)
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].str.replace('river_3p6', '72', regex=False)
    
    df_games_turns_formatted['turn_id'] = df_games_turns_formatted['turn_id'].astype(int)

    # sort by game ID and turn ID
    df_games_turns_formatted = df_games_turns_formatted.sort_values(
        by=['game_id', 'turn_id'],
        ascending=[True, True]
    )

    # Update the order of the columns
    new_order = ['game_id', 'turn_id', 'player_id', 'player_name', 'stage', 'sub_stage', 'cards', 'flop', 'turn', 'river', 
                 'nb_players_left', 'nb_previous_raise', 'action', 'amount_raised', 'amount_required', 'amount_committed', 'amount_finish']
    df_games_turns_formatted = df_games_turns_formatted[new_order]

    return df_games_turns_formatted

## Function chen_score
This function gets only the 2 starting cards as an input and return a calculated chen score. It uses the Chen formula

In [9]:
# Chen Formula for Texas Hold'em starting hand evaluation
# This program calculates the Chen score for any two-card starting hand.
RANK_VALUES = {
    'A': 10,  # Ace
    'K': 8,
    'Q': 7,
    'J': 6,
    'T': 5,   # Ten
    '9': 4.5,
    '8': 4,
    '7': 3.5,
    '6': 3,
    '5': 2.5,
    '4': 2,
    '3': 1.5,
    '2': 1
}

def chen_score(card1: str, card2: str) -> float:
    """
    This function gets only the 2 starting cards as an input
    and return a calculated chen score. It uses the Chen formula : https://en.wikipedia.org/wiki/Texas_hold_%27em_starting_hands
    
    Arguments
    ---------
    c1: string
           Card 1
    c2: string
           Card 2

    Returns
    --------
    score: float
         Number which represent the strength of the cards analyzed in the input. The higher the number is the stronger the 2 cards are
    """
    try:
        # Extract rank and suit
        rank1, suit1 = card1[0].upper(), card1[1].lower()
        rank2, suit2 = card2[0].upper(), card2[1].lower()

        if rank1 not in RANK_VALUES or rank2 not in RANK_VALUES:
            raise ValueError("Invalid card rank.")
        if suit1 not in "shdc" or suit2 not in "shdc":
            raise ValueError("Invalid card suit.")

        # Step 1: Base score = highest card value
        high_rank, low_rank = (rank1, rank2) if RANK_VALUES[rank1] >= RANK_VALUES[rank2] else (rank2, rank1)
        score = RANK_VALUES[high_rank]

        # Step 2: Pair bonus
        if rank1 == rank2:
            score *= 2
            if score < 5:
                score = 5  # Minimum for low pairs

        # Step 3: Suited bonus
        if suit1 == suit2:
            score += 2

        # Step 4: Gap penalty
        gap = abs(list(RANK_VALUES.keys()).index(high_rank) - list(RANK_VALUES.keys()).index(low_rank)) - 1
        if gap == 1:
            score -= 1
        elif gap == 2:
            score -= 2
        elif gap == 3:
            score -= 4
        elif gap >= 4:
            score -= 5

        # Step 5: Small gap + connectedness bonus
        if gap <= 1 and RANK_VALUES[low_rank] >= 5:  # 5 or higher
            score += 1

        return round(max(score, 0), 1)  # No negative scores

    except Exception as e:
        print(f"Error: {e}")
        return 0.0

## Function cards_score
This function gets a minimum of 2 cards and a maximum of 7 cards as an input and return a calculated card score. The lower the card score is, the better.

In [10]:
def cards_score(c1: str, c2: str, c3: str, c4: str, c5: str, c6: str, c7: str):
    """
    This function gets a minimum of 2 cards and a maximum of 7 cards as an input
    and return a calculated card score. The lower the card score is, the better.
    
    Arguments
    ---------
    c1: string
           Card 1
    c2: string
           Card 2
    c3: string
           Card 3
    c4: string
           Card 4
    c5: string
           Card 5
    c6: string
           Card 6
    c7: string
           Card 7

    Returns
    --------
    cards_score_result: float
         Number which represent the strength of the cards analyzed in the input. The lower the number is the stronger the hands of cards are.
    """
    cards_score_result = 0

    if (len(c1) > 0) & (len(c2) > 0) & (len(c3) == 0) & (len(c4) == 0) & (len(c5) == 0) & (len(c6) == 0) & (len(c7) == 0):
        # find a color not in the list of the first 2 cards
        list_colors = ['d', 'h', 's', 'c']
        c1_color = c1[1:2]
        c2_color = c2[1:2]
        list_colors.remove(c1_color)
        if c1_color != c2_color:
            list_colors.remove(c2_color)
        color_pick = list_colors[0]
        # find 3 other small cards to add to the initial 2 cards
        list_cards = ['2', '3', '4', '5', '6', '7']
        temp_cards_score_result = 0
        max_cards_score_result = 0
        for card3 in list_cards:
            for card4 in list_cards:
                for card5 in list_cards:
                    try:
                        temp_cards_score_result = evaluate_cards(c1, c2, card3 + color_pick, card4 + color_pick, card5 + color_pick)
                        if temp_cards_score_result >= max_cards_score_result:
                            max_cards_score_result = temp_cards_score_result
                    except Exception:
                        pass
        cards_score_result = max_cards_score_result
    elif (len(c1) > 0) & (len(c2) > 0) & (len(c3) > 0) & (len(c4) > 0) & (len(c5) > 0) & (len(c6) == 0) & (len(c7) == 0):
        cards_score_result = evaluate_cards(c1, c2, c3, c4, c5)
    elif (len(c1) > 0) & (len(c2) > 0) & (len(c3) > 0) & (len(c4) > 0) & (len(c5) > 0) & (len(c6) > 0) & (len(c7) == 0):
        cards_score_result = evaluate_cards(c1, c2, c3, c4, c5, c6)
    elif (len(c1) > 0) & (len(c2) > 0) & (len(c3) > 0) & (len(c4) > 0) & (len(c5) > 0) & (len(c6) > 0) & (len(c7) > 0):
        cards_score_result = evaluate_cards(c1, c2, c3, c4, c5, c6, c7)
    
    return cards_score_result

## Function enrichTurnsDataframe
This function reads a dataframe of Poker Games aggregated by Game ID, Player ID and Turn ID and enrich it with multiple additional columns

In [22]:
def enrichTurnsDataframe(df_games_turns):
    """
    This function reads a dataframe of Poker Games aggregated by Game ID, Player ID and Turn ID
    and enrich it with multiple additional columns
    
    Arguments
    ---------
    df_games_turns: pd.DataFrame
           Array aggregated by Game ID, Player ID and Turn ID.

    Returns
    --------
    df_games_turns_enriched: pd.DataFrame
         Array aggregated by poker Game ID, Player ID and Turn ID. It was enriched with multiple additional columns
    """

    # add column total_cards to show the list of cards available
    df_games_turns_enriched = df_games_turns.reset_index()
    df_games_turns_enriched.loc[df_games_turns_enriched['stage'] == 'preflop', 'total_cards'] = df_games_turns_enriched['cards']
    df_games_turns_enriched.loc[df_games_turns_enriched['stage'] == 'flop', 'total_cards'] = df_games_turns_enriched['cards'] + df_games_turns_enriched['flop']
    df_games_turns_enriched.loc[df_games_turns_enriched['stage'] == 'turn', 'total_cards'] = df_games_turns_enriched['cards'] + df_games_turns_enriched['flop'] + df_games_turns_enriched['turn']
    df_games_turns_enriched.loc[df_games_turns_enriched['stage'] == 'river', 'total_cards'] = df_games_turns_enriched['cards'] + df_games_turns_enriched['flop'] + df_games_turns_enriched['turn'] + df_games_turns_enriched['river']

    # add a column c1, c2, c3, c4, c5, c6, c7 which show each card
    # c1 and c2 are the 2 cards belonging to the player only
    # c3, c4, c5, c6, and c7 are the community cards
    df_games_turns_enriched['c1'] = df_games_turns_enriched['total_cards'].astype(str).str[0:2]
    df_games_turns_enriched['c2'] = df_games_turns_enriched['total_cards'].astype(str).str[2:4]
    df_games_turns_enriched['c3'] = df_games_turns_enriched['total_cards'].astype(str).str[4:6]
    df_games_turns_enriched['c4'] = df_games_turns_enriched['total_cards'].astype(str).str[6:8]
    df_games_turns_enriched['c5'] = df_games_turns_enriched['total_cards'].astype(str).str[8:10]
    df_games_turns_enriched['c6'] = df_games_turns_enriched['total_cards'].astype(str).str[10:12]
    df_games_turns_enriched['c7'] = df_games_turns_enriched['total_cards'].astype(str).str[12:14]

    #add column chen_score
    df_games_turns_enriched['chen_score'] = 0
    for index, row in df_games_turns_enriched.iterrows():
        c1 = df_games_turns_enriched['c1'].iloc[index]
        c2 = df_games_turns_enriched['c2'].iloc[index]
        if (len(c1) > 0) & (len(c2) > 0):
            df_games_turns_enriched.loc[index, "chen_score"] = chen_score(c1, c2)
            
    #add column cards_score
    df_games_turns_enriched['cards_score'] = 0
    for index, row in df_games_turns_enriched.iterrows():
        c1 = df_games_turns_enriched['c1'].iloc[index]
        c2 = df_games_turns_enriched['c2'].iloc[index]
        c3 = df_games_turns_enriched['c3'].iloc[index]
        c4 = df_games_turns_enriched['c4'].iloc[index]
        c5 = df_games_turns_enriched['c5'].iloc[index]
        c6 = df_games_turns_enriched['c6'].iloc[index]
        c7 = df_games_turns_enriched['c7'].iloc[index]
        df_games_turns_enriched.loc[index, "cards_score"] = cards_score(c1, c2, c3, c4, c5, c6, c7)

    #add column pct_score
    df_games_turns_enriched['pct_score'] = (df_games_turns_enriched['amount_finish'] - 10000) / 10000

    # update some columns to datatype int
    df_games_turns_enriched['cards_score'] = df_games_turns_enriched['cards_score'].astype(int)

    # add column action int
    df_games_turns_enriched['action_int'] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['action'] == 'C'),'action_int'] = 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['action'] == 'R'),'action_int'] = 2
    
    # add column action flag
    df_games_turns_enriched['action_flag'] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['action'] == 'C'),'action_flag'] = 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['action'] == 'R'),'action_flag'] = 1

    # add column position_int
    df_games_turns_enriched['position_int'] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p1', 'position_int'] = 1
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p2', 'position_int'] = 2
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p3', 'position_int'] = 3
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p4', 'position_int'] = 4
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p5', 'position_int'] = 5
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p6', 'position_int'] = 6
    df_games_turns_enriched['position_int'] = df_games_turns_enriched['position_int'].astype(int)
    
    # add column position code 1
    df_games_turns_enriched['position_code_1'] = 'none'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p1', 'position_code_1'] = 'SB'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p2', 'position_code_1'] = 'BB'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p3', 'position_code_1'] = 'UTG'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p4', 'position_code_1'] = 'MP'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p5', 'position_code_1'] = 'CO'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p6', 'position_code_1'] = 'BTN'
    
    # add column position code 2
    df_games_turns_enriched['position_code_2'] = 'none'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p1', 'position_code_2'] = 'EP'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p2', 'position_code_2'] = 'EP'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p3', 'position_code_2'] = 'MP'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p4', 'position_code_2'] = 'MP'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p5', 'position_code_2'] = 'LP'
    df_games_turns_enriched.loc[df_games_turns_enriched['player_id'] == 'p6', 'position_code_2'] = 'LP'
    
    # update column sub_stage
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'preflop_1', 'sub_stage'] = '1'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'preflop_2', 'sub_stage'] = '2'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'preflop_3', 'sub_stage'] = '3'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'flop_1', 'sub_stage'] = '1'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'flop_2', 'sub_stage'] = '2'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'flop_3', 'sub_stage'] = '3'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'turn_1', 'sub_stage'] = '1'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'turn_2', 'sub_stage'] = '2'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'turn_3', 'sub_stage'] = '3'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'river_1', 'sub_stage'] = '1'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'river_2', 'sub_stage'] = '2'
    df_games_turns_enriched.loc[df_games_turns_enriched['sub_stage'] == 'river_3', 'sub_stage'] = '3'
    
    df_games_turns_enriched['sub_stage'] = df_games_turns_enriched['sub_stage'].astype(int)

    # add column nb_flush_player_cards
    df_games_turns_enriched['clubs_count'] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[1:2] == 'c', 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[3:4] == 'c', 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    
    df_games_turns_enriched['spades_count'] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[1:2] == 's', 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[3:4] == 's', 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    
    df_games_turns_enriched['hearts_count'] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[1:2] == 'h', 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[3:4] == 'h', 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    
    df_games_turns_enriched['diamonds_count'] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[1:2] == 'd', 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[df_games_turns_enriched['cards'].astype(str).str[3:4] == 'd', 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    
    df_games_turns_enriched['nb_flush_player_cards'] = df_games_turns_enriched[['clubs_count', 'spades_count', 'hearts_count', 'diamonds_count']].max(axis=1)
    
    df_games_turns_enriched = df_games_turns_enriched.drop(columns=['clubs_count', 'spades_count', 'hearts_count', 'diamonds_count'])
    
    # if the number of flush cards is greater than 5 we let it be at 5
    df_games_turns_enriched.loc[df_games_turns_enriched['nb_flush_player_cards'] > 5, 'nb_flush_player_cards'] = 5

    # add column nb_straight_player_cards
    df_games_turns_enriched['c1s'] = df_games_turns_enriched['c1'].astype(str).str[0:1]
    df_games_turns_enriched['c2s'] = df_games_turns_enriched['c2'].astype(str).str[0:1]
    df_games_turns_enriched['c3s'] = df_games_turns_enriched['c3'].astype(str).str[0:1]
    df_games_turns_enriched['c4s'] = df_games_turns_enriched['c4'].astype(str).str[0:1]
    df_games_turns_enriched['c5s'] = df_games_turns_enriched['c5'].astype(str).str[0:1]
    df_games_turns_enriched['c6s'] = df_games_turns_enriched['c6'].astype(str).str[0:1]
    df_games_turns_enriched['c7s'] = df_games_turns_enriched['c7'].astype(str).str[0:1]
    
    # Bucket 1-5
    bucket_field_name = 'bucket_A_5_count'
    bucket_list = ['A', '2', '3', '4', '5']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 2-6
    bucket_field_name = 'bucket_2_6_count'
    bucket_list = ['2', '3', '4', '5', '6']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 3-7
    bucket_field_name = 'bucket_3_7_count'
    bucket_list = ['3', '4', '5', '6', '7']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 4-8
    bucket_field_name = 'bucket_4_8_count'
    bucket_list = ['4', '5', '6', '7', '8']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 5-9
    bucket_field_name = 'bucket_5_9_count'
    bucket_list = ['5', '6', '7', '8', '9']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 6-T
    bucket_field_name = 'bucket_6_T_count'
    bucket_list = ['6', '7', '8', '9', 'T']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 7-J
    bucket_field_name = 'bucket_7_J_count'
    bucket_list = ['7', '8', '9', 'T', 'J']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 8-Q
    bucket_field_name = 'bucket_8_Q_count'
    bucket_list = ['8', '9', 'T', 'J', 'Q']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 9-K
    bucket_field_name = 'bucket_9_K_count'
    bucket_list = ['9', 'T', 'J', 'Q', 'K']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket T-A
    bucket_field_name = 'bucket_T_A_count'
    bucket_list = ['T', 'J', 'Q', 'K', 'A']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[df_games_turns_enriched['c1s'].isin(bucket_list), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['c2s'].isin(bucket_list)) & (df_games_turns_enriched['c1s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    df_games_turns_enriched['nb_straight_player_cards'] = df_games_turns_enriched[['bucket_A_5_count', 'bucket_2_6_count', 'bucket_3_7_count', 'bucket_4_8_count', 'bucket_5_9_count', 'bucket_6_T_count', 'bucket_7_J_count', 'bucket_8_Q_count', 'bucket_9_K_count', 'bucket_T_A_count']].max(axis=1)
    df_games_turns_enriched = df_games_turns_enriched.drop(columns=['bucket_A_5_count', 'bucket_2_6_count', 'bucket_3_7_count', 'bucket_4_8_count', 'bucket_5_9_count', 'bucket_6_T_count', 'bucket_7_J_count', 'bucket_8_Q_count', 'bucket_9_K_count', 'bucket_T_A_count'])

    # add column nb_flush_community_cards
    df_games_turns_enriched['clubs_count'] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 'c'), 'clubs_count'] = df_games_turns_enriched['clubs_count'] + 1
    
    df_games_turns_enriched['spades_count'] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 's'), 'spades_count'] = df_games_turns_enriched['spades_count'] + 1
    
    df_games_turns_enriched['hearts_count'] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 'h'), 'hearts_count'] = df_games_turns_enriched['hearts_count'] + 1
    
    df_games_turns_enriched['diamonds_count'] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[1:2] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[3:4] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['flop'].astype(str).str[5:6] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['turn'].astype(str).str[1:2] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['river'].astype(str).str[1:2] == 'd'), 'diamonds_count'] = df_games_turns_enriched['diamonds_count'] + 1
    
    df_games_turns_enriched['nb_flush_community_cards'] = df_games_turns_enriched[['clubs_count', 'spades_count', 'hearts_count', 'diamonds_count']].max(axis=1)
    df_games_turns_enriched = df_games_turns_enriched.drop(columns=['clubs_count', 'spades_count', 'hearts_count', 'diamonds_count'])

    # add column nb_straight_community_cards
    df_games_turns_enriched['c1s'] = df_games_turns_enriched['c1'].astype(str).str[0:1]
    df_games_turns_enriched['c2s'] = df_games_turns_enriched['c2'].astype(str).str[0:1]
    df_games_turns_enriched['c3s'] = df_games_turns_enriched['c3'].astype(str).str[0:1]
    df_games_turns_enriched['c4s'] = df_games_turns_enriched['c4'].astype(str).str[0:1]
    df_games_turns_enriched['c5s'] = df_games_turns_enriched['c5'].astype(str).str[0:1]
    df_games_turns_enriched['c6s'] = df_games_turns_enriched['c6'].astype(str).str[0:1]
    df_games_turns_enriched['c7s'] = df_games_turns_enriched['c7'].astype(str).str[0:1]
    
    # Bucket 1-5
    bucket_field_name = 'bucket_A_5_count'
    bucket_list = ['A', '2', '3', '4', '5']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 2-6
    bucket_field_name = 'bucket_2_6_count'
    bucket_list = ['2', '3', '4', '5', '6']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 3-7
    bucket_field_name = 'bucket_3_7_count'
    bucket_list = ['3', '4', '5', '6', '7']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 4-8
    bucket_field_name = 'bucket_4_8_count'
    bucket_list = ['4', '5', '6', '7', '8']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 5-9
    bucket_field_name = 'bucket_5_9_count'
    bucket_list = ['5', '6', '7', '8', '9']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 6-T
    bucket_field_name = 'bucket_6_T_count'
    bucket_list = ['6', '7', '8', '9', 'T']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 7-J
    bucket_field_name = 'bucket_7_J_count'
    bucket_list = ['7', '8', '9', 'T', 'J']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 8-Q
    bucket_field_name = 'bucket_8_Q_count'
    bucket_list = ['8', '9', 'T', 'J', 'Q']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket 9-K
    bucket_field_name = 'bucket_9_K_count'
    bucket_list = ['9', 'T', 'J', 'Q', 'K']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    # Bucket T-A
    bucket_field_name = 'bucket_T_A_count'
    bucket_list = ['T', 'J', 'Q', 'K', 'A']
    df_games_turns_enriched[bucket_field_name] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c3s'].isin(bucket_list)) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c3s'] != df_games_turns_enriched['c2s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1 
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c4s'].isin(bucket_list)) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c4s'] != df_games_turns_enriched['c3s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['flop', 'turn', 'river'])) & (df_games_turns_enriched['c5s'].isin(bucket_list)) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c5s'] != df_games_turns_enriched['c4s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['turn', 'river'])) & (df_games_turns_enriched['c6s'].isin(bucket_list)) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c6s'] != df_games_turns_enriched['c5s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'].isin(['river'])) & (df_games_turns_enriched['c7s'].isin(bucket_list)) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c1s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c2s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c3s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c4s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c5s']) & (df_games_turns_enriched['c7s'] != df_games_turns_enriched['c6s']), bucket_field_name] = df_games_turns_enriched[bucket_field_name] + 1
    
    df_games_turns_enriched['nb_straight_community_cards'] = df_games_turns_enriched[['bucket_A_5_count', 'bucket_2_6_count', 'bucket_3_7_count', 'bucket_4_8_count', 'bucket_5_9_count', 'bucket_6_T_count', 'bucket_7_J_count', 'bucket_8_Q_count', 'bucket_9_K_count', 'bucket_T_A_count']].max(axis=1)
    df_games_turns_enriched = df_games_turns_enriched.drop(columns=['bucket_A_5_count', 'bucket_2_6_count', 'bucket_3_7_count', 'bucket_4_8_count', 'bucket_5_9_count', 'bucket_6_T_count', 'bucket_7_J_count', 'bucket_8_Q_count', 'bucket_9_K_count', 'bucket_T_A_count'])

    # add column stage_int
    df_games_turns_enriched['stage_int'] = 0
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'preflop') & (df_games_turns_enriched['sub_stage'] == 1), 'stage_int'] = 1
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'preflop') & (df_games_turns_enriched['sub_stage'] == 2), 'stage_int'] = 2
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'preflop') & (df_games_turns_enriched['sub_stage'] == 3), 'stage_int'] = 3
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'flop') & (df_games_turns_enriched['sub_stage'] == 1), 'stage_int'] = 4
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'flop') & (df_games_turns_enriched['sub_stage'] == 2), 'stage_int'] = 5
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'flop') & (df_games_turns_enriched['sub_stage'] == 3), 'stage_int'] = 6
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'turn') & (df_games_turns_enriched['sub_stage'] == 1), 'stage_int'] = 7
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'turn') & (df_games_turns_enriched['sub_stage'] == 2), 'stage_int'] = 8
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'turn') & (df_games_turns_enriched['sub_stage'] == 3), 'stage_int'] = 9
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'river') & (df_games_turns_enriched['sub_stage'] == 1), 'stage_int'] = 10
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'river') & (df_games_turns_enriched['sub_stage'] == 2), 'stage_int'] = 11
    df_games_turns_enriched.loc[(df_games_turns_enriched['stage'] == 'river') & (df_games_turns_enriched['sub_stage'] == 3), 'stage_int'] = 12
    
    # remove columns which are not necessary
    df_games_turns_enriched = df_games_turns_enriched.drop(columns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7'])
    df_games_turns_enriched = df_games_turns_enriched.drop(columns=['c1s', 'c2s', 'c3s', 'c4s', 'c5s', 'c6s', 'c7s'])
    df_games_turns_enriched = df_games_turns_enriched.drop(columns=['total_cards', 'index'])

    # reset the index
    df_games_turns_enriched = df_games_turns_enriched.reset_index()

    # Update the order of the columns
    new_order = ['index', 'game_id', 'turn_id', 'player_id', 'player_name', 'stage', 'sub_stage', 'stage_int', 'position_int', 'position_code_1', 'position_code_2', 
                 'cards', 'flop', 'turn', 'river', 
                 'action', 'action_int', 'action_flag', 
                 'amount_raised', 'amount_required', 'amount_committed', 'amount_finish', 
                 'nb_players_left', 'nb_previous_raise', 'nb_straight_player_cards', 'nb_straight_community_cards', 'nb_flush_player_cards', 'nb_flush_community_cards', 
                 'chen_score', 'cards_score', 'pct_score']
    df_games_turns_enriched = df_games_turns_enriched[new_order]

    return df_games_turns_enriched

# Create the preprocessed CSV file
We want to create a CSV preprocessed file which will show the action performed during each turn by each player for all the 10000 poker games

In [12]:
# We call the function readFolderPHH to read from the folder where are located all the 10000 Poker Hand History files
read_folder = readFolderPHH(folder_path)

In [13]:
# We combine all the dataframes returned after we called the previous function readFolderPHH into only one dataframe for simplicity
df_games_temp1 = pd.merge(left = read_folder[0], right = read_folder[1], left_on = 'game_id', right_on = 'game_id', how = 'inner')
df_games_temp2 = pd.merge(left = df_games_temp1, right = read_folder[2], left_on = 'game_id', right_on = 'game_id', how = 'inner')
df_games_temp3 = pd.merge(left = df_games_temp2, right = read_folder[3], left_on = 'game_id', right_on = 'game_id', how = 'inner')
df_games = pd.merge(left = df_games_temp3, right = read_folder[4], left_on = 'game_id', right_on = 'game_id', how = 'inner')

df_games.head()

,game_id,variant,ante_trimming_status,p1_ante,p2_ante,p3_ante,p4_ante,p5_ante,p6_ante,p1_blind_or_straddle,...,p6_river_action_2,p1_river_action_2,p2_river_action_2,p3_river_action_3,p4_river_action_3,p5_river_action_3,p6_river_action_3,p1_river_action_3,p2_river_action_3,river_start_player
0,99__99,NT,true,0,0,0,0,0,0,50,...,,,f,,,,,,,p2
1,99__98,NT,true,0,0,0,0,0,0,50,...,,,,,,,,,,p2
2,99__97,NT,true,0,0,0,0,0,0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99__96,NT,true,0,0,0,0,0,0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99__95,NT,true,0,0,0,0,0,0,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# In this dataframe df_games, we have 10000 rows : one for each of the 10000 poker games player between 6 players.
df_games.describe()

,game_id,variant,ante_trimming_status,p1_ante,p2_ante,p3_ante,p4_ante,p5_ante,p6_ante,p1_blind_or_straddle,...,p6_river_action_2,p1_river_action_2,p2_river_action_2,p3_river_action_3,p4_river_action_3,p5_river_action_3,p6_river_action_3,p1_river_action_3,p2_river_action_3,river_start_player
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,2748,2748,2748,2748,2748,2748,2748,2748,2748,2656
unique,10000,1,1,1,1,1,1,1,1,1,...,5,26,40,1,1,1,1,2,3,5
top,99__99,NT,true,0,0,0,0,0,0,50,...,,,,,,,,,,p2
freq,1,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,2726,2437,2313,2748,2748,2748,2748,2747,2745,1318


In [15]:
# We want to create another dataframe which shows more clearly the actions performed by each player during each game
# We want a dataframe where the key fields will be Game ID and Player ID
# this code runs in about 1 minute
df_games_players = getDataframeByGameIDPlayerID(df_games)
df_games_players.head()

,game_id,small_blind,big_blind,min_bet,starting_stack,player_name,player_id,sm,finishing_stack,d_dh,...,turn_d_db,turn_start_player,turn_action_1,turn_action_2,turn_action_3,river_d_db,river_start_player,river_action_1,river_action_2,river_action_3
0,100__0,50,100,100,10000,MrBlue,p1,False,10310,TcQc,...,7c,p1,cc,,,Qh,p1,cbr 230,,
1,100__0,50,100,100,10000,MrBlonde,p2,False,9900,8s4c,...,7c,p1,,,,Qh,p1,,,
2,100__0,50,100,100,10000,MrWhite,p3,False,10000,9c3d,...,7c,p1,,,,Qh,p1,,,
3,100__0,50,100,100,10000,MrPink,p4,False,9790,Ah4h,...,7c,p1,cc,,,Qh,p1,f,,
4,100__0,50,100,100,10000,MrBrown,p5,False,10000,Th5s,...,7c,p1,,,,Qh,p1,,,


In [16]:
# In this dataframe df_games_players, we have 60000 rows : 6 rows for each of the 10000 poker games played between 6 players.
# There is one row per player for each game
df_games_players.describe()

,game_id,small_blind,big_blind,min_bet,starting_stack,player_name,player_id,sm,finishing_stack,d_dh,...,turn_d_db,turn_start_player,turn_action_1,turn_action_2,turn_action_3,river_d_db,river_start_player,river_action_1,river_action_2,river_action_3
count,60000,60000,60000,60000,60000,60000,60000,60000,60000,60000,...,23124,22686,23124,23124,23124,16488,15936,16488,16488,16488
unique,10000,1,1,1,1,14,6,2,1399,2652,...,52,5,378,69,3,52,5,525,76,3
top,100__0,50,100,100,10000,Pluribus,p1,False,10000,8cQs,...,9h,p2,,,,Ah,p2,,,
freq,6,60000,60000,60000,60000,10000,10000,56598,31109,43,...,564,11346,15270,21788,23120,396,7908,11050,15545,16484


In [17]:
# We want to add multiple columns to the dataframe df_games_players to get additional detailed information
df_games_players_enriched = enrichPlayersDataframe(df_games_players)

In [18]:
# We want to create another dataframe which shows more clearly the actions performed during each turn by each player during each game
# We want a dataframe where the key fields will be Game ID, Turn ID
# this code runs in about 45 minutes
df_games_turns = getDataframeByGameIDTurnID(df_games_players_enriched)
df_games_turns.head()

,game_id,small_blind,big_blind,min_bet,starting_stack,player_name,player_id,sm,finishing_stack,d_dh,...,river_d_db,stage,sub_stage,turn_id,action,action_raise_amount,nb_previous_raise,nb_players_left,amount_already_in,amount_to_continue
12,100__0,50,100,100,10000,MrBlue,p1,False,10310,TcQc,...,Qh,preflop,preflop_1,1,cc,0,0,3.0,50,160
13,100__0,50,100,100,10000,MrBlue,p1,False,10310,TcQc,...,Qh,flop,flop_1,4,cc,0,0,2.0,210,0
14,100__0,50,100,100,10000,MrBlue,p1,False,10310,TcQc,...,Qh,turn,turn_1,7,cc,0,0,2.0,210,0
15,100__0,50,100,100,10000,MrBlue,p1,False,10310,TcQc,...,Qh,river,river_1,10,cbr,230,0,2.0,210,0
16,100__0,50,100,100,10000,MrBlonde,p2,False,9900,8s4c,...,Qh,preflop,preflop_1,1,f,0,0,3.0,100,110


In [19]:
# In this dataframe df_games_turns, we have 91356 rows : one row for each turn and action played by each of the players during the 10000 poker games.
df_games_turns.describe()

,small_blind,big_blind,turn_id,action_raise_amount,nb_previous_raise,nb_players_left,amount_already_in,amount_to_continue
count,91356.0,91356.0,91356.000000,91356.000000,91356.000000,91356.000000,91356.000000,91356.000000
mean,50.0,100.0,2.783353,135.331724,0.240685,3.392552,197.140987,212.983132
std,0.0,0.0,2.852808,591.217607,0.530209,1.456352,450.109140,550.229692
min,50.0,100.0,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,50.0,100.0,1.000000,0.000000,0.000000,2.000000,0.000000,50.000000
50%,50.0,100.0,1.000000,0.000000,0.000000,3.000000,50.000000,100.000000
75%,50.0,100.0,4.000000,0.000000,0.000000,5.000000,225.000000,200.000000
max,50.0,100.0,12.000000,9900.000000,4.000000,6.000000,9499.000000,9800.000000


In [20]:
# We want to update the formatting of the dataframe df_games_turns : We remove some columns, rename other columns and update the sorting
df_games_turns_formatted = formatTurnsDataframe(df_games_turns)

In [23]:
# We want to add additional calculated columns to the dataframe df_games_turns
df_games_turns_enriched = enrichTurnsDataframe(df_games_turns_formatted)

In [24]:
# We save the preprocessed data file to the directory data/csv/
df_games_turns_enriched.to_csv('data/poker_games.csv', index=False)

# Summary of the final preprocessed CSV file
Description of the columns:
- index: File index (Integer)
- game_id : Game ID (String)
- turn_id : Turn ID (Integer). It is a number. It relates to the order of the actions that were played during one game. The first player to play will start to play during turn_id = 1. The last move will be played with the highest turn_id for that particular game
- player_id : Player ID (String). As there are 6 players, it should be in the list: p1, p2, p3, p4, p5, and p6.
- player_name : Player Name (String). The name of the player
- stage : Stage of the game (String). It should be in the list : preflop, flop, turn or river
- sub_stage : Sub stage of the game (Integer). When a player makes a first move during a stage of the game, the sub_stage will be 1. If there is a raise by another player, and he needs to play again during the same stage of the game, then the sub_stage number will increase by 1. In the dataset, the maximum sub_stage is 3
- stage_int : Stage of the game (Integer). A numeric representation of the stage and sub_stage of the game. 1 corresponds to stage Preflop, sub_stage 1. 2 corresponds to stage Preflop, sub_stage 2,... and 12 corresponds to stage River, sub_stage 3. Note that in each stage, the maximum sub_stage is 3.
- position_int : Position (Integer). It should be a number between 1 and 6 as there are 6 players. It is the position of the player at the table. 1 is for Small Blind, 2 is for Big Blind, 3 is for Under the Gun, 4 is for Middle Position, 5 is for Cut-off and 6 is for Button
- position_code_1 : Position Code 1 (String). It should be a string in the list: SB, BB, UTG, MP, CO, BTN. It is the position of the player at the table. SB is for Small Blind, BB is for Big Blind, UTG is for Under the Gun, MP is for Middle Position, CO is for Cut-off and BTN is for Button
- position_code_2 : Position Code 2 (String). It should be a string in the list: EP, MP, LP. It is the position of the player at the table. EP is for Early Position, MP is for Middle Position and LP is for Late Position.
- cards : Cards (String). The 2 cards given to the player at the start of the game. The first character is for the first card and is in the list A, 2, 3, 4, 5, 6, 7, 8, 9, T, J, Q, K. (Note that T is a code for 10). The second character is the suit for the first card and is in the list c, d, h, s (c for clubs, d for diamonds, h for hearts and s for spades. The third and fourth characters follow similar logic but are for the second card
- flop : Flop (String). The 3 community cards that players can see during the flop stage.
- turn : Turn (String). The additional community card that players can see during the turn stage.
- river : River (String). The additional community card that players can see during the river stage.
- action : Action (String). The action played by the player during the turn. It should be in the list F, C, R (F for Fold, C for Call and R for Raise)
- action_int : Action (Integer). The action played by the player during the turn. It should be in the list 0, 1, 2 (0 for Fold, 1 for Call and 2 for Raise)
- action_flag : Action (Boolean). The action played by the player during the turn. It should be in the list 0, 1, (0 for Fold, 1 for Call or Raise)
- amount_raised : Amount raised (Integer). The amount raised by the player during the turn. As the starting stack is 10000, this number should be between 0 and 10000
- amount_required : Amount required (Integer). The amount which was raised previously by another player and which is required by the current player to continue playing the game. If the amount is greater than 0, it means another player has previously raised. As the starting stack is 10000, this number should be between 0 and 10000
- amount_committed : Amount committed (Integer). The amount which was previously already committed by the player into the pot. As the starting stack is 10000, this number should be between 0 and 10000
- amount_finish : Amount finish (Float). The amount that the player ends up with at the end of the current game.
- nb_players_left : Number of players left (Integer). As there are 6 players for each game, this number should be between 1 and 6. 
- nb_previous_raise : Number of previous raises (Integer). The number of times that the current player has already made a raise during this game.
- nb_straight_player_cards : Number of connected cards that could potentially make a straight (Integer). When the number reaches 5, the player has a straight. This includes the 2 cards from the player and the community cards
- nb_straight_community_cards : Number of connected cards that could potentially make a straight (Integer). When the number reaches 5, the player has a straight. This includes only the community cards and not the 2 cards from the player
- nb_flush_player_cards : Number of connected cards that could potentially make a flush (Integer). When the number reaches 5, the player has a flush. This includes the 2 cards from the player and the community cards
- nb_flush_community_cards : Number of connected cards that could potentially make a flush (Integer). When the number reaches 5, the player has a flush. This includes only the community cards and not the 2 cards from the player
- chen_score : Chen score from the 2 starting cards of the player (Float). It is the result of the Chen formula described at https://en.wikipedia.org/wiki/Texas_hold_%27em_starting_hands. The higher the number is, the stronger the starting hand of the player is
- cards_score : Cards score (Float). It is an evaluation of the strength of the 2 starting cards of the player combined with the community cards available. The result is a number between 1 and 8000. The lower the number is, the stronger the hand of the player is.
- pct__score : Percentage score (Float). This is a percentage indicator of how well the player did at the end of the game. It is calculated using field amount_finish with the formula (amount_finish - 10000) / 10000

In [25]:
df_games_turns_enriched.head()

,index,game_id,turn_id,player_id,player_name,stage,sub_stage,stage_int,position_int,position_code_1,...,amount_finish,nb_players_left,nb_previous_raise,nb_straight_player_cards,nb_straight_community_cards,nb_flush_player_cards,nb_flush_community_cards,chen_score,cards_score,pct_score
0,0,100_0,1,p3,MrWhite,preflop,1,1,3,UTG,...,10000.0,6,0,1,0,1,0,0.0,7444,0.000
1,1,100_0,2,p4,MrPink,preflop,1,1,4,MP,...,9790.0,5,0,2,0,2,0,7.0,6678,-0.021
2,2,100_0,3,p5,MrBrown,preflop,1,1,5,CO,...,10000.0,5,0,1,0,1,0,0.0,7410,0.000
3,3,100_0,4,p6,Pluribus,preflop,1,1,6,BTN,...,10000.0,4,0,2,0,1,0,3.5,7461,0.000
4,4,100_0,5,p1,MrBlue,preflop,1,1,1,SB,...,10310.0,3,0,2,0,2,0,9.0,7146,0.031
